This Google Colab aims to replicate our results with the BLURB leaderboard. This colab uses JAX hugging face code. We connected this Google colab to the local machine (our TPUv3-8 device). To connect your TPU VM machine with google colab follow this page: https://research.google.com/colaboratory/local-runtimes.html.

Part of our code is modified from BioLinkBERT Github repo : https://github.com/michiyasunaga/LinkBERT.

Please also note that you can run this code also on GPU without any further modification. However, running the code with the given parameters here in this colab may not reproduce the same results on the GPU since here we use TPU. Thus you need to change the hyperparameters choices. See this post for more detail: https://github.com/google/jax/issues/4823

Also note that you can run codes inside each cell directly on the TPU VM machine with Jax installed to reproduce our results. If you do not want to log results no need to connect to the local machine.

1. first step lets download the pre-processed BLURB dataset from the BioLinkBERT team GitHub :

In [ ]:
!wget https://nlp.stanford.edu/projects/myasu/LinkBERT/data.zip
!unzip data.zip

2. run both [run_flax_seq.py, run_flax_ner.py] cells to write the code for each file in our machine. This code uses wandb to log each run. Our project name is "blurb-flax". you can change it from lines 43-44:

```
import wandb
wandb.init(project="blurb-flax")
```

You can disabled wandb by running :

`!wandb disabled`



In [ ]:
#@title run_flax_seq.py
%%writefile run_flax_seq.py
#!/usr/bin/env python
# coding=utf-8
# Copyright 2021 The HuggingFace Inc. team. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
""" Finetuning a 🤗 Flax Transformers model for sequence classification on GLUE."""
import json
import logging
import math
import os
import random
import sys
import time
from dataclasses import dataclass, field
from pathlib import Path
from typing import Any, Callable, Dict, Optional, Tuple

import datasets
import evaluate
import jax
import jax.numpy as jnp
import numpy as np
import optax
from datasets import load_dataset
from flax import struct, traverse_util
from flax.jax_utils import pad_shard_unpad, replicate, unreplicate
from flax.training import train_state
from flax.training.common_utils import get_metrics, onehot, shard
from huggingface_hub import Repository, create_repo
from tqdm import tqdm
import wandb
wandb.init(project="blurb-flax")
import transformers
from transformers import (
    AutoConfig,
    AutoTokenizer,
    FlaxAutoModelForSequenceClassification,
    HfArgumentParser,
    PretrainedConfig,
    TrainingArguments,
    is_tensorboard_available,
)
from transformers.utils import check_min_version, get_full_repo_name, send_example_telemetry


logger = logging.getLogger(__name__)
# Will error if the minimal version of Transformers is not installed. Remove at your own risks.
check_min_version("4.27.0.dev0")

Array = Any
Dataset = datasets.arrow_dataset.Dataset
PRNGKey = Any

task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}


@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    use_slow_tokenizer: Optional[bool] = field(
        default=False,
        metadata={"help": "If passed, will use a slow tokenizer (not backed by the 🤗 Tokenizers library)."},
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where do you want to store the pretrained models downloaded from huggingface.co"},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "The specific model version to use (can be a branch name, tag name or commit id)."},
    )
    use_auth_token: bool = field(
        default=False,
        metadata={
            "help": (
                "Will use the token generated when running `huggingface-cli login` (necessary to use this script "
                "with private models)."
            )
        },
    )


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    task_name: Optional[str] = field(
        default=None, metadata={"help": f"The name of the glue task to train on. choices {list(task_to_keys.keys())}"}
    )
    dataset_config_name: Optional[str] = field(
        default=None, metadata={"help": "The configuration name of the dataset to use (via the datasets library)."}
    )
    train_file: Optional[str] = field(
        default=None, metadata={"help": "The input training data file (a csv or JSON file)."}
    )
    validation_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input evaluation data file to evaluate on (a csv or JSON file)."},
    )
    test_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input test data file to predict on (a csv or JSON file)."},
    )
    text_column_name: Optional[str] = field(
        default=None, metadata={"help": "The column name of text to input in the file (a csv or JSON file)."}
    )
    label_column_name: Optional[str] = field(
        default=None, metadata={"help": "The column name of label to input in the file (a csv or JSON file)."}
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    max_seq_length: int = field(
        default=None,
        metadata={
            "help": (
                "The maximum total input sequence length after tokenization. If set, sequences longer "
                "than this will be truncated, sequences shorter will be padded."
            )
        },
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "For debugging purposes or quicker training, truncate the number of training examples to this "
                "value if set."
            )
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
                "value if set."
            )
        },
    )
    max_predict_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "For debugging purposes or quicker training, truncate the number of prediction examples to this "
                "value if set."
            )
        },
    )
    blurb_task: Optional[str] = field(
        default=None, metadata={"help": f"The name of the glue task to train on. choices {list(task_to_keys.keys())}"}
    )
    metric_name: Optional[str] = field(
        default=None,
        metadata={"help": "The name of the metric"},
    )

    def __post_init__(self):
        if self.task_name is None and self.train_file is None and self.validation_file is None:
            raise ValueError("Need either a dataset name or a training/validation file.")
        else:
            if self.train_file is not None:
                extension = self.train_file.split(".")[-1]
                assert extension in ["csv", "json"], "`train_file` should be a csv or a json file."
            if self.validation_file is not None:
                extension = self.validation_file.split(".")[-1]
                assert extension in ["csv", "json"], "`validation_file` should be a csv or a json file."
        self.task_name = self.task_name.lower() if type(self.task_name) == str else self.task_name


def create_train_state(
    model: FlaxAutoModelForSequenceClassification,
    learning_rate_fn: Callable[[int], float],
    is_regression: bool,
    num_labels: int,
    weight_decay: float,
) -> train_state.TrainState:
    """Create initial training state."""

    class TrainState(train_state.TrainState):
        """Train state with an Optax optimizer.

        The two functions below differ depending on whether the task is classification
        or regression.

        Args:
          logits_fn: Applied to last layer to obtain the logits.
          loss_fn: Function to compute the loss.
        """

        logits_fn: Callable = struct.field(pytree_node=False)
        loss_fn: Callable = struct.field(pytree_node=False)

    # We use Optax's "masking" functionality to not apply weight decay
    # to bias and LayerNorm scale parameters. decay_mask_fn returns a
    # mask boolean with the same structure as the parameters.
    # The mask is True for parameters that should be decayed.
    def decay_mask_fn(params):
        flat_params = traverse_util.flatten_dict(params)
        # find out all LayerNorm parameters
        layer_norm_candidates = ["layernorm", "layer_norm", "ln"]
        layer_norm_named_params = set(
            [
                layer[-2:]
                for layer_norm_name in layer_norm_candidates
                for layer in flat_params.keys()
                if layer_norm_name in "".join(layer).lower()
            ]
        )
        flat_mask = {path: (path[-1] != "bias" and path[-2:] not in layer_norm_named_params) for path in flat_params}
        return traverse_util.unflatten_dict(flat_mask)

    tx = optax.adamw(
        learning_rate=learning_rate_fn, b1=0.9, b2=0.999, eps=1e-6, weight_decay=weight_decay, mask=decay_mask_fn
    )

    if is_regression:

        def mse_loss(logits, labels):
            return jnp.mean((logits[..., 0] - labels) ** 2)

        return TrainState.create(
            apply_fn=model.__call__,
            params=model.params,
            tx=tx,
            logits_fn=lambda logits: logits[..., 0],
            loss_fn=mse_loss,
        )
    else:  # Classification.

        def cross_entropy_loss(logits, labels):
            xentropy = optax.softmax_cross_entropy(logits, onehot(labels, num_classes=num_labels))
            return jnp.mean(xentropy)

        return TrainState.create(
            apply_fn=model.__call__,
            params=model.params,
            tx=tx,
            logits_fn=lambda logits: logits.argmax(-1),
            loss_fn=cross_entropy_loss,
        )


def create_learning_rate_fn(
    train_ds_size: int, train_batch_size: int, num_train_epochs: int, num_warmup_steps: int, learning_rate: float
) -> Callable[[int], jnp.array]:
    """Returns a linear warmup, linear_decay learning rate function."""
    steps_per_epoch = train_ds_size // train_batch_size
    num_train_steps = steps_per_epoch * num_train_epochs
    warmup_fn = optax.linear_schedule(init_value=0.0, end_value=learning_rate, transition_steps=num_warmup_steps)
    decay_fn = optax.linear_schedule(
        init_value=learning_rate, end_value=0, transition_steps=num_train_steps - num_warmup_steps
    )
    schedule_fn = optax.join_schedules(schedules=[warmup_fn, decay_fn], boundaries=[num_warmup_steps])
    return schedule_fn


def glue_train_data_collator(rng: PRNGKey, dataset: Dataset, batch_size: int):
    """Returns shuffled batches of size `batch_size` from truncated `train dataset`, sharded over all local devices."""
    steps_per_epoch = len(dataset) // batch_size
    perms = jax.random.permutation(rng, len(dataset))
    perms = perms[: steps_per_epoch * batch_size]  # Skip incomplete batch.
    perms = perms.reshape((steps_per_epoch, batch_size))

    for perm in perms:
        batch = dataset[perm]
        batch = {k: np.array(v) for k, v in batch.items()}
        batch = shard(batch)

        yield batch


def glue_eval_data_collator(dataset: Dataset, batch_size: int):
    """Returns batches of size `batch_size` from `eval dataset`. Sharding handled by `pad_shard_unpad` in the eval loop."""
    batch_idx = np.arange(len(dataset))

    steps_per_epoch = math.ceil(len(dataset) / batch_size)
    batch_idx = np.array_split(batch_idx, steps_per_epoch)

    for idx in batch_idx:
        batch = dataset[idx]
        batch = {k: np.array(v) for k, v in batch.items()}

        yield batch


def main():
    parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
    if len(sys.argv) == 2 and sys.argv[1].endswith(".json"):
        # If we pass only one argument to the script and it's the path to a json file,
        # let's parse it to get our arguments.
        model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath(sys.argv[1]))
    else:
        model_args, data_args, training_args = parser.parse_args_into_dataclasses()

    wandb.log({"blurb_task":data_args.blurb_task,"model_name":model_args.model_name_or_path,
    "per_device_train_batch_size":training_args.per_device_train_batch_size,
    "learning_rate": training_args.learning_rate, "weight_decay": training_args.weight_decay,
    "adam_beta1":training_args.adam_beta1, "adam_beta2" : training_args.adam_beta2,
    "adam_epsilon":training_args.adam_epsilon,"label_smoothing_factor": training_args.label_smoothing_factor,
     "num_train_epochs":training_args.num_train_epochs, "warmup_steps":training_args.warmup_steps,"seed": training_args.seed})

    # Sending telemetry. Tracking the example usage helps us better allocate resources to maintain them. The
    # information sent is the one passed as arguments along with your Python/PyTorch versions.
    send_example_telemetry("run_glue", model_args, data_args, framework="flax")

    # Make one log on every process with the configuration for debugging.
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
    )
    # Setup logging, we only want one process per machine to log things on the screen.
    logger.setLevel(logging.INFO if jax.process_index() == 0 else logging.ERROR)
    if jax.process_index() == 0:
        datasets.utils.logging.set_verbosity_warning()
        transformers.utils.logging.set_verbosity_info()
    else:
        datasets.utils.logging.set_verbosity_error()
        transformers.utils.logging.set_verbosity_error()

    # Handle the repository creation
    if training_args.push_to_hub:
        if training_args.hub_model_id is None:
            repo_name = get_full_repo_name(
                Path(training_args.output_dir).absolute().name, token=training_args.hub_token
            )
        else:
            repo_name = training_args.hub_model_id
        create_repo(repo_name, exist_ok=True, token=training_args.hub_token)
        repo = Repository(training_args.output_dir, clone_from=repo_name, token=training_args.hub_token)

    # Get the datasets: you can either provide your own CSV/JSON training and evaluation files (see below)
    # or specify a GLUE benchmark task (the dataset will be downloaded automatically from the datasets Hub).

    # For CSV/JSON files, this script will use as labels the column called 'label' and as pair of sentences the
    # sentences in columns called 'sentence1' and 'sentence2' if such column exists or the first two columns not named
    # label if at least two columns are provided.

    # If the CSVs/JSONs contain only one non-label column, the script does single sentence classification on this
    # single column. You can easily tweak this behavior (see below)

    # In distributed training, the load_dataset function guarantee that only one local process can concurrently
    # download the dataset.
    if data_args.task_name is not None:
        # Downloading and loading a dataset from the hub.
        raw_datasets = load_dataset(
            "glue",
            data_args.task_name,
            use_auth_token=True if model_args.use_auth_token else None,
        )
    else:
        # Loading the dataset from local csv or json file.
        data_files = {}
        if data_args.train_file is not None:
            data_files["train"] = data_args.train_file
        if data_args.validation_file is not None:
            data_files["validation"] = data_args.validation_file
        if data_args.test_file is not None:
            data_files["test"] = data_args.test_file
        extension = (data_args.train_file if data_args.train_file is not None else data_args.valid_file).split(".")[-1]
        raw_datasets = load_dataset(
            extension,
            data_files=data_files,
            use_auth_token=True if model_args.use_auth_token else None,
        )
    # See more about loading any type of standard or custom dataset at
    # https://huggingface.co/docs/datasets/loading_datasets.html.

    # Labels
    if data_args.task_name is not None:
        is_regression = data_args.task_name == "stsb"
        if not is_regression:
            label_list = raw_datasets["train"].features["label"].names
            num_labels = len(label_list)
        else:
            num_labels = 1
    else:
        # Trying to have good defaults here, don't hesitate to tweak to your needs.
        is_regression = raw_datasets["train"].features["label"].dtype in ["float32", "float64"]
        if is_regression:
            num_labels = 1
        else:
            # A useful fast method:
            # https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.unique
            label_list = raw_datasets["train"].unique("label")
            label_list.sort()  # Let's sort it for determinism
            num_labels = len(label_list)

    # Load pretrained model and tokenizer
    config = AutoConfig.from_pretrained(
        model_args.model_name_or_path,
        num_labels=num_labels,
        finetuning_task=data_args.task_name,
        use_auth_token=True if model_args.use_auth_token else None,
    )
    tokenizer = AutoTokenizer.from_pretrained(
        model_args.model_name_or_path,
        use_fast=not model_args.use_slow_tokenizer,
        use_auth_token=True if model_args.use_auth_token else None,
    )
    model = FlaxAutoModelForSequenceClassification.from_pretrained(
        model_args.model_name_or_path,
        config=config,
        use_auth_token=True if model_args.use_auth_token else None,
        from_pt=True
    )

    # Preprocessing the datasets
    if data_args.task_name is not None:
        sentence1_key, sentence2_key = task_to_keys[data_args.task_name]
    else:
        # Again, we try to have some nice defaults but don't hesitate to tweak to your use case.
        non_label_column_names = [name for name in raw_datasets["train"].column_names if name != "label"]
        if "sentence1" in non_label_column_names and "sentence2" in non_label_column_names:
            sentence1_key, sentence2_key = "sentence1", "sentence2"
        elif "sentence" in non_label_column_names:
            sentence1_key, sentence2_key = "sentence", None
        else:
            if len(non_label_column_names) >= 2:
                sentence1_key, sentence2_key = non_label_column_names[:2]
            else:
                sentence1_key, sentence2_key = non_label_column_names[0], None

    # Some models have set the order of the labels to use, so let's make sure we do use it.
    label_to_id = None
    id_to_label=None
    if (
        model.config.label2id != PretrainedConfig(num_labels=num_labels).label2id
        and data_args.task_name is not None
        and not is_regression
    ):
        # Some have all caps in their config, some don't.
        label_name_to_id = {k.lower(): v for k, v in model.config.label2id.items()}
        if list(sorted(label_name_to_id.keys())) == list(sorted(label_list)):
            logger.info(
                f"The configuration of the model provided the following label correspondence: {label_name_to_id}. "
                "Using it!"
            )
            label_to_id = {i: label_name_to_id[label_list[i]] for i in range(num_labels)}
        else:
            logger.warning(
                "Your model seems to have been trained with labels, but they don't match the dataset: ",
                f"model labels: {list(sorted(label_name_to_id.keys()))}, dataset labels: {list(sorted(label_list))}."
                "\nIgnoring the model labels as a result.",
            )
    elif data_args.task_name is None and is_regression==False:
        label_to_id = {v: i for i, v in enumerate(label_list)}

    if not is_regression:
     id_to_label={v: i for  v,i in enumerate(label_to_id)}
    if "chemprot" in str(data_args.train_file).lower():
     id_to_label[0]="false"
    if "ddi" in str(data_args.train_file).lower():
     id_to_label[0]="DDI-false"
    def preprocess_function(examples):
        # Tokenize the texts
        texts = (
            (examples[sentence1_key],) if sentence2_key is None else (examples[sentence1_key], examples[sentence2_key])
        )
        result = tokenizer(*texts, padding="max_length", max_length=data_args.max_seq_length, truncation=True)

        if "label" in examples:
            if label_to_id is not None:
                # Map labels to IDs (not necessary for GLUE tasks)
                result["labels"] = [label_to_id[l] for l in examples["label"]]
            else:
                # In all cases, rename the column to labels because the model will expect that.
                result["labels"] = examples["label"]
        return result

    processed_datasets = raw_datasets.map(
        preprocess_function, batched=True, remove_columns=raw_datasets["train"].column_names
    )

    train_dataset = processed_datasets["train"]
    eval_dataset = processed_datasets["validation_matched" if data_args.task_name == "mnli" else "validation"]
    test_dataset = processed_datasets["test_matched" if data_args.task_name == "mnli" else "test"]
    # Log a few random samples from the training set:
    for index in random.sample(range(len(train_dataset)), 3):
        logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

    # Define a summary writer
    has_tensorboard = is_tensorboard_available()
    if has_tensorboard and jax.process_index() == 0:
        try:
            from flax.metrics.tensorboard import SummaryWriter

            summary_writer = SummaryWriter(training_args.output_dir)
            summary_writer.hparams({**training_args.to_dict(), **vars(model_args), **vars(data_args)})
        except ImportError as ie:
            has_tensorboard = False
            logger.warning(
                f"Unable to display metrics through TensorBoard because some package are not installed: {ie}"
            )
    else:
        logger.warning(
            "Unable to display metrics through TensorBoard because the package is not installed: "
            "Please run pip install tensorboard to enable."
        )
    def compute_metrics(preds, labels):
          # Get the metric function
        if data_args.task_name is not None:
          metric = evaluate.load("glue", data_args.task_name)
        else:
          metric = evaluate.load("accuracy")

        if data_args.metric_name == "hoc":
            from utils_hoc import eval_hoc
            labels = np.array(p.label_ids).astype(int) #[num_ex, num_class]
            preds = (np.array(preds) > 0).astype(int)  #[num_ex, num_class]
            ids = eval_dataset["id"]
            return eval_hoc(labels.tolist(), preds.tolist(), list(ids))
        if data_args.task_name is not None:
            result = metric.compute(predictions=preds, references=labels)
            if len(result) > 1:
                result["combined_score"] = np.mean(list(result.values())).item()
            return result
        elif data_args.metric_name == "pearsonr":
            from scipy.stats import pearsonr as scipy_pearsonr
            pearsonr = float(scipy_pearsonr(labels, preds)[0])
            return {"pearsonr": pearsonr}
        elif data_args.metric_name == "PRF1":
            TP = ((preds == labels) & (preds != 0)).astype(int).sum().item()
            P_total = (preds != 0).astype(int).sum().item()
            L_total = (labels != 0).astype(int).sum().item()
            P = TP / P_total if P_total else 0
            R = TP / L_total if L_total else 0
            F1 = 2 * P * R / (P + R) if (P + R) else 0
            #confirm=precision_recall_fscore_support(labels, preds, average='micro',labels=[1])
            return {"precision": P, "recall": R, "F1": F1}
        elif is_regression:
            return {"mse": ((preds - labels) ** 2).mean().item()}
        else:
            return {"accuracy": (preds == labels).astype(np.float32).mean().item()}

    def write_train_metric(summary_writer, train_metrics, train_time, step):
        summary_writer.scalar("train_time", train_time, step)

        train_metrics = get_metrics(train_metrics)
        for key, vals in train_metrics.items():
            tag = f"train_{key}"
            for i, val in enumerate(vals):
                summary_writer.scalar(tag, val, step - len(vals) + i + 1)

    def write_eval_metric(summary_writer, eval_metrics, step):
        for metric_name, value in eval_metrics.items():
            summary_writer.scalar(f"eval_{metric_name}", value, step)

    num_epochs = int(training_args.num_train_epochs)
    rng = jax.random.PRNGKey(training_args.seed)
    dropout_rngs = jax.random.split(rng, jax.local_device_count())

    train_batch_size = int(training_args.per_device_train_batch_size) * jax.local_device_count()
    per_device_eval_batch_size = int(training_args.per_device_eval_batch_size)
    eval_batch_size = per_device_eval_batch_size * jax.device_count()

    learning_rate_fn = create_learning_rate_fn(
        len(train_dataset),
        train_batch_size,
        training_args.num_train_epochs,
        training_args.warmup_steps,
        training_args.learning_rate,
    )

    state = create_train_state(
        model, learning_rate_fn, is_regression, num_labels=num_labels, weight_decay=training_args.weight_decay
    )

    # define step functions
    def train_step(
        state: train_state.TrainState, batch: Dict[str, Array], dropout_rng: PRNGKey
    ) -> Tuple[train_state.TrainState, float]:
        """Trains model with an optimizer (both in `state`) on `batch`, returning a pair `(new_state, loss)`."""
        dropout_rng, new_dropout_rng = jax.random.split(dropout_rng)
        targets = batch.pop("labels")

        def loss_fn(params):
            logits = state.apply_fn(**batch, params=params, dropout_rng=dropout_rng, train=True)[0]
            loss = state.loss_fn(logits, targets)
            return loss

        grad_fn = jax.value_and_grad(loss_fn)
        loss, grad = grad_fn(state.params)
        grad = jax.lax.pmean(grad, "batch")
        new_state = state.apply_gradients(grads=grad)
        metrics = jax.lax.pmean({"loss": loss, "learning_rate": learning_rate_fn(state.step)}, axis_name="batch")
        return new_state, metrics, new_dropout_rng

    p_train_step = jax.pmap(train_step, axis_name="batch", donate_argnums=(0,))

    def eval_step(state, batch):
        logits = state.apply_fn(**batch, params=state.params, train=False)[0]
        return state.logits_fn(logits)

    p_eval_step = jax.pmap(eval_step, axis_name="batch")

    if data_args.task_name is not None:
        metric = evaluate.load("glue", data_args.task_name)
    else:
        metric = evaluate.load("accuracy")

    logger.info(f"===== Starting training ({num_epochs} epochs) =====")
    train_time = 0

    # make sure weights are replicated on each device
    state = replicate(state)

    steps_per_epoch = len(train_dataset) // train_batch_size
    total_steps = steps_per_epoch * num_epochs
    epochs = tqdm(range(num_epochs), desc=f"Epoch ... (0/{num_epochs})", position=0, leave=True)
    for epoch in epochs:
        train_start = time.time()
        train_metrics = []

        # Create sampling rng
        rng, input_rng = jax.random.split(rng)

        # train
        train_loader = glue_train_data_collator(input_rng, train_dataset, train_batch_size)
        for step, batch in enumerate(
            tqdm(
                train_loader,
                total=steps_per_epoch,
                desc="Training...",
                position=0, leave=True,
            ),
        ):
            state, train_metric, dropout_rngs = p_train_step(state, batch, dropout_rngs)
            train_metrics.append(train_metric)

            cur_step = (epoch * steps_per_epoch) + (step + 1)

            if cur_step % training_args.logging_steps == 0 and cur_step > 0:
                # Save metrics
                train_metric = unreplicate(train_metric)
                train_time += time.time() - train_start
                if has_tensorboard and jax.process_index() == 0:
                    write_train_metric(summary_writer, train_metrics, train_time, cur_step)

                epochs.write(
                    f"Step... ({cur_step}/{total_steps} | Training Loss: {train_metric['loss']}, Learning Rate:"
                    f" {train_metric['learning_rate']})"
                )

                train_metrics = []

            if (cur_step % training_args.eval_steps == 0 or cur_step % steps_per_epoch == 0) and cur_step > 0:
                # evaluate
                eval_loader = glue_eval_data_collator(eval_dataset, eval_batch_size)
                preds_list=[]
                labels_list=[]
                for batch in tqdm(
                    eval_loader,
                    total=math.ceil(len(eval_dataset) / eval_batch_size),
                    desc="Evaluating ...",
                    position=0, leave=True,
                ):
                    labels = batch.pop("labels")
                    predictions = pad_shard_unpad(p_eval_step)(
                        state, batch, min_device_batch=per_device_eval_batch_size
                    )
                    for pred_example in predictions:
                      preds_list.append(pred_example)
                    for label_example in labels:
                      labels_list.append(label_example)
                    metric.add_batch(predictions=np.array(predictions), references=labels)

                preds_list=np.array(preds_list)
                labels_list=np.array(labels_list)
                blurb_result=compute_metrics(preds_list,labels_list)
                blurb_result={"eval_" + str(key): val for key, val in blurb_result.items()}
                wandb.log(blurb_result)
                eval_metric = metric.compute()
                logger.info(f"{blurb_result}| Step... ({cur_step}/{total_steps} | Eval metrics: {eval_metric}) ")

                if has_tensorboard and jax.process_index() == 0:
                    write_eval_metric(summary_writer, eval_metric, cur_step)
            if 7==0:
            #if (cur_step % training_args.save_steps == 0 and cur_step > 0) or (cur_step == total_steps):
                # save checkpoint after each epoch and push checkpoint to the hub
                if jax.process_index() == 0:
                    params = jax.device_get(unreplicate(state.params))
                    model.save_pretrained(training_args.output_dir, params=params)
                    tokenizer.save_pretrained(training_args.output_dir)
                    if training_args.push_to_hub:
                        repo.push_to_hub(commit_message=f"Saving weights and logs of step {cur_step}", blocking=False)
            epochs.desc = f"Epoch ... {epoch + 1}/{num_epochs}"

    # save the eval metrics in json
    if data_args.test_file:
                ########### the only change here from dev code is that we pass "test_dataset" to glue_eval_data_collator
                eval_loader = glue_eval_data_collator(test_dataset, eval_batch_size)
                preds_list=[]
                preds_dict={}
                labels_list=[]
                for batch in tqdm(
                    eval_loader,
                    total=math.ceil(len(test_dataset) / eval_batch_size),
                    desc="Evaluating on Test Data ...",
                     position=0, leave=True
                ):
                    labels = batch.pop("labels")
                    predictions = pad_shard_unpad(p_eval_step)(
                        state, batch, min_device_batch=per_device_eval_batch_size
                    )
                    ##### prepare predictions and labels to metric compute function ####
                    for pred_example in predictions:
                      preds_list.append(pred_example)
                    for label_example in labels:
                      labels_list.append(label_example)
                    metric.add_batch(predictions=np.array(predictions), references=labels)
                    ####################################################################
                ############ write prediction in BLURB Leaderboard Format ##########
                raw_ids=[]
                for row in raw_datasets["test"]:
                 raw_ids.append(row["id"])
                for label_id,pred_value in zip(raw_ids,preds_list):
                      if is_regression:
                        preds_dict[label_id]=str(pred_value)
                      else:
                        preds_dict[label_id]=str(id_to_label[pred_value])
                if "/" in model_args.model_name_or_path:
                   model_name_hf=model_args.model_name_or_path.split("/")[-1]
                else:
                   model_name_hf=model_args.model_name_or_path
                final_entry={"dataset_name": data_args.blurb_task, "model_name":model_name_hf, "predictions":preds_dict}
                with open(model_args.model_name_or_path.split("/")[-1]+"_"+str(data_args.blurb_task)+".json", "w") as f:
                  json.dump(final_entry, f, indent=4, sort_keys=True)
                #####################################################################
                preds_list=np.array(preds_list)
                labels_list=np.array(labels_list)
                blurb_result=compute_metrics(preds_list,labels_list)
                blurb_result={"test_" + str(key): val for key, val in blurb_result.items()}
                wandb.log(blurb_result)
                eval_metric = metric.compute()
                logger.info(f" test results : {blurb_result}")
    if jax.process_index() == 0:
        eval_metric = {f"eval_{metric_name}": value for metric_name, value in eval_metric.items()}
        path = os.path.join(training_args.output_dir, "eval_results.json")
        with open(path, "w") as f:
            json.dump(eval_metric, f, indent=4, sort_keys=True)


if __name__ == "__main__":
    main()

Writing run_flax_seq.py


In [ ]:
#@title run_flax_ner.py
%%writefile run_flax_ner.py
#!/usr/bin/env python
# coding=utf-8
# Copyright 2021 The HuggingFace Inc. team. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
""" Fine-tuning a 🤗 Flax Transformers model on token classification tasks (NER, POS, CHUNKS)"""
import json
import logging
import math
import os
import random
import sys
import time
from dataclasses import asdict, dataclass, field
from enum import Enum
from itertools import chain
from pathlib import Path
from typing import Any, Callable, Dict, Optional, Tuple

import datasets
import numpy as np
from datasets import ClassLabel, load_dataset
from tqdm import tqdm

import evaluate
import jax
import jax.numpy as jnp
import optax
import transformers
from flax import struct, traverse_util
from flax.jax_utils import pad_shard_unpad, replicate, unreplicate
from flax.training import train_state
from flax.training.common_utils import get_metrics, onehot, shard
from huggingface_hub import Repository
from transformers import (
    AutoConfig,
    AutoTokenizer,
    FlaxAutoModelForTokenClassification,
    HfArgumentParser,
    is_tensorboard_available,
)
from transformers.utils import check_min_version, get_full_repo_name, send_example_telemetry
from transformers.utils.versions import require_version


logger = logging.getLogger(__name__)
# Will error if the minimal version of Transformers is not installed. Remove at your own risks.
check_min_version("4.26.0.dev0")

require_version("datasets>=1.8.0", "To fix: pip install -r examples/pytorch/token-classification/requirements.txt")

Array = Any
Dataset = datasets.arrow_dataset.Dataset
PRNGKey = Any
import wandb
wandb.init(project="blurb-flax")
wandb.init()

@dataclass
class TrainingArguments:
    output_dir: str = field(
        metadata={"help": "The output directory where the model predictions and checkpoints will be written."},
    )
    overwrite_output_dir: bool = field(
        default=False,
        metadata={
            "help": (
                "Overwrite the content of the output directory. "
                "Use this to continue training if output_dir points to a checkpoint directory."
            )
        },
    )
    do_train: bool = field(default=False, metadata={"help": "Whether to run training."})
    do_eval: bool = field(default=False, metadata={"help": "Whether to run eval on the dev set."})
    do_test: bool = field(default=False, metadata={"help": "Whether to run eval on the test set."})
    per_device_train_batch_size: int = field(
        default=8, metadata={"help": "Batch size per GPU/TPU core/CPU for training."}
    )
    per_device_eval_batch_size: int = field(
        default=8, metadata={"help": "Batch size per GPU/TPU core/CPU for evaluation."}
    )
    learning_rate: float = field(default=5e-5, metadata={"help": "The initial learning rate for AdamW."})
    weight_decay: float = field(default=0.0, metadata={"help": "Weight decay for AdamW if we apply some."})
    adam_beta1: float = field(default=0.9, metadata={"help": "Beta1 for AdamW optimizer"})
    adam_beta2: float = field(default=0.999, metadata={"help": "Beta2 for AdamW optimizer"})
    adam_epsilon: float = field(default=1e-8, metadata={"help": "Epsilon for AdamW optimizer."})
    adafactor: bool = field(default=False, metadata={"help": "Whether or not to replace AdamW by Adafactor."})
    num_train_epochs: float = field(default=3.0, metadata={"help": "Total number of training epochs to perform."})
    warmup_steps: int = field(default=0, metadata={"help": "Linear warmup over warmup_steps."})
    logging_steps: int = field(default=500, metadata={"help": "Log every X updates steps."})
    save_steps: int = field(default=500, metadata={"help": "Save checkpoint every X updates steps."})
    eval_steps: int = field(default=None, metadata={"help": "Run an evaluation every X steps."})
    seed: int = field(default=42, metadata={"help": "Random seed that will be set at the beginning of training."})
    push_to_hub: bool = field(
        default=False, metadata={"help": "Whether or not to upload the trained model to the model hub after training."}
    )
    hub_model_id: str = field(
        default=None, metadata={"help": "The name of the repository to keep in sync with the local `output_dir`."}
    )
    hub_token: str = field(default=None, metadata={"help": "The token to use to push to the Model Hub."})

    def __post_init__(self):
        if self.output_dir is not None:
            self.output_dir = os.path.expanduser(self.output_dir)

    def to_dict(self):
        """
        Serializes this instance while replace `Enum` by their values (for JSON serialization support). It obfuscates
        the token values by removing their value.
        """
        d = asdict(self)
        for k, v in d.items():
            if isinstance(v, Enum):
                d[k] = v.value
            if isinstance(v, list) and len(v) > 0 and isinstance(v[0], Enum):
                d[k] = [x.value for x in v]
            if k.endswith("_token"):
                d[k] = f"<{k.upper()}>"
        return d


@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where do you want to store the pretrained models downloaded from huggingface.co"},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "The specific model version to use (can be a branch name, tag name or commit id)."},
    )
    use_auth_token: bool = field(
        default=False,
        metadata={
            "help": (
                "Will use the token generated when running `huggingface-cli login` (necessary to use this script "
                "with private models)."
            )
        },
    )


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    task_name: Optional[str] = field(default="ner", metadata={"help": "The name of the task (ner, pos...)."})
    dataset_name: Optional[str] = field(
        default=None, metadata={"help": "The name of the dataset to use (via the datasets library)."}
    )
    dataset_config_name: Optional[str] = field(
        default=None, metadata={"help": "The configuration name of the dataset to use (via the datasets library)."}
    )
    train_file: Optional[str] = field(
        default=None, metadata={"help": "The input training data file (a csv or JSON file)."}
    )
    validation_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input evaluation data file to evaluate on (a csv or JSON file)."},
    )
    test_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input test data file to predict on (a csv or JSON file)."},
    )
    text_column_name: Optional[str] = field(
        default=None, metadata={"help": "The column name of text to input in the file (a csv or JSON file)."}
    )
    label_column_name: Optional[str] = field(
        default=None, metadata={"help": "The column name of label to input in the file (a csv or JSON file)."}
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    max_seq_length: int = field(
        default=None,
        metadata={
            "help": (
                "The maximum total input sequence length after tokenization. If set, sequences longer "
                "than this will be truncated, sequences shorter will be padded."
            )
        },
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "For debugging purposes or quicker training, truncate the number of training examples to this "
                "value if set."
            )
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
                "value if set."
            )
        },
    )
    max_predict_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "For debugging purposes or quicker training, truncate the number of prediction examples to this "
                "value if set."
            )
        },
    )
    label_all_tokens: bool = field(
        default=False,
        metadata={
            "help": (
                "Whether to put the label for one word on all tokens of generated by that word or just on the "
                "one (in which case the other tokens will have a padding index)."
            )
        },
    )
    return_entity_level_metrics: bool = field(
        default=False,
        metadata={"help": "Whether to return all the entity levels during evaluation or just the overall ones."},
    )
    return_macro_metrics: bool = field(
        default=False,
        metadata={"help": "Whether to return macro metrics (i.e. final mean is taken across tag categories) during evaluation or just the overall ones (i.e. micro)."},
    )
    blurb_task: Optional[str] = field(
        default=None, metadata={"help": "The column name of text to input in the file (a csv or JSON file)."}
    )

    def __post_init__(self):
        if self.dataset_name is None and self.train_file is None and self.validation_file is None:
            raise ValueError("Need either a dataset name or a training/validation file.")
        else:
            if self.train_file is not None:
                extension = self.train_file.split(".")[-1]
                assert extension in ["csv", "json"], "`train_file` should be a csv or a json file."
            if self.validation_file is not None:
                extension = self.validation_file.split(".")[-1]
                assert extension in ["csv", "json"], "`validation_file` should be a csv or a json file."
        self.task_name = self.task_name.lower()


def create_train_state(
    model: FlaxAutoModelForTokenClassification,
    learning_rate_fn: Callable[[int], float],
    num_labels: int,
    training_args: TrainingArguments,
) -> train_state.TrainState:
    """Create initial training state."""

    class TrainState(train_state.TrainState):
        """Train state with an Optax optimizer.

        The two functions below differ depending on whether the task is classification
        or regression.

        Args:
          logits_fn: Applied to last layer to obtain the logits.
          loss_fn: Function to compute the loss.
        """

        logits_fn: Callable = struct.field(pytree_node=False)
        loss_fn: Callable = struct.field(pytree_node=False)

    # We use Optax's "masking" functionality to not apply weight decay
    # to bias and LayerNorm scale parameters. decay_mask_fn returns a
    # mask boolean with the same structure as the parameters.
    # The mask is True for parameters that should be decayed.
    def decay_mask_fn(params):
        flat_params = traverse_util.flatten_dict(params)
        # find out all LayerNorm parameters
        layer_norm_candidates = ["layernorm", "layer_norm", "ln"]
        layer_norm_named_params = set(
            [
                layer[-2:]
                for layer_norm_name in layer_norm_candidates
                for layer in flat_params.keys()
                if layer_norm_name in "".join(layer).lower()
            ]
        )
        flat_mask = {path: (path[-1] != "bias" and path[-2:] not in layer_norm_named_params) for path in flat_params}
        return traverse_util.unflatten_dict(flat_mask)

    tx = optax.adamw(
        learning_rate=learning_rate_fn,
        b1=training_args.adam_beta1,
        b2=training_args.adam_beta2,
        eps=training_args.adam_epsilon,
        weight_decay=training_args.weight_decay,
        mask=decay_mask_fn,
    )

    def cross_entropy_loss(logits, labels):
        xentropy = optax.softmax_cross_entropy(logits, onehot(labels, num_classes=num_labels))
        return jnp.mean(xentropy)

    return TrainState.create(
        apply_fn=model.__call__,
        params=model.params,
        tx=tx,
        logits_fn=lambda logits: logits.argmax(-1),
        loss_fn=cross_entropy_loss,
    )


def create_learning_rate_fn(
    train_ds_size: int, train_batch_size: int, num_train_epochs: int, num_warmup_steps: int, learning_rate: float
) -> Callable[[int], jnp.array]:
    """Returns a linear warmup, linear_decay learning rate function."""
    steps_per_epoch = train_ds_size // train_batch_size
    num_train_steps = steps_per_epoch * num_train_epochs
    warmup_fn = optax.linear_schedule(init_value=0.0, end_value=learning_rate, transition_steps=num_warmup_steps)
    decay_fn = optax.linear_schedule(
        init_value=learning_rate, end_value=0, transition_steps=num_train_steps - num_warmup_steps
    )
    schedule_fn = optax.join_schedules(schedules=[warmup_fn, decay_fn], boundaries=[num_warmup_steps])
    return schedule_fn


def train_data_collator(rng: PRNGKey, dataset: Dataset, batch_size: int):
    """Returns shuffled batches of size `batch_size` from truncated `train dataset`, sharded over all local devices."""
    steps_per_epoch = len(dataset) // batch_size
    perms = jax.random.permutation(rng, len(dataset))
    perms = perms[: steps_per_epoch * batch_size]  # Skip incomplete batch.
    perms = perms.reshape((steps_per_epoch, batch_size))

    for perm in perms:
        batch = dataset[perm]
        batch = {k: np.array(v,) for k, v in batch.items()}
        batch = shard(batch)
        yield batch


def eval_data_collator(dataset: Dataset, batch_size: int):
    """Returns batches of size `batch_size` from `eval dataset`. Sharding handled by `pad_shard_unpad` in the eval loop."""
    batch_idx = np.arange(len(dataset))

    steps_per_epoch = math.ceil(len(dataset) / batch_size)
    batch_idx = np.array_split(batch_idx, steps_per_epoch)

    for idx in batch_idx:
        batch = dataset[idx]
        batch = {k: np.array(v) for k, v in batch.items()}
        yield batch


def main():
    # See all possible arguments in src/transformers/training_args.py
    # or by passing the --help flag to this script.
    # We now keep distinct sets of args, for a cleaner separation of concerns.

    parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
    if len(sys.argv) == 2 and sys.argv[1].endswith(".json"):
        # If we pass only one argument to the script and it's the path to a json file,
        # let's parse it to get our arguments.
        model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath(sys.argv[1]))
    else:
        model_args, data_args, training_args = parser.parse_args_into_dataclasses()
    wandb.log({"blurb_task":data_args.blurb_task,"model_name":model_args.model_name_or_path,"per_device_train_batch_size":training_args.per_device_train_batch_size, "learning_rate": training_args.learning_rate, "num_train_epochs":training_args.num_train_epochs, "warmup_steps":training_args.warmup_steps,"seed": training_args.seed})
    # Sending telemetry. Tracking the example usage helps us better allocate resources to maintain them. The
    # information sent is the one passed as arguments along with your Python/PyTorch versions.
    send_example_telemetry("run_ner", model_args, data_args, framework="flax")

    # Make one log on every process with the configuration for debugging.
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
    )
    # Setup logging, we only want one process per machine to log things on the screen.
    logger.setLevel(logging.INFO if jax.process_index() == 0 else logging.ERROR)
    if jax.process_index() == 0:
        datasets.utils.logging.set_verbosity_warning()
        transformers.utils.logging.set_verbosity_info()
    else:
        datasets.utils.logging.set_verbosity_error()
        transformers.utils.logging.set_verbosity_error()

    # Handle the repository creation
    if training_args.push_to_hub:
        if training_args.hub_model_id is None:
            repo_name = get_full_repo_name(
                Path(training_args.output_dir).absolute().name, token=training_args.hub_token
            )
        else:
            repo_name = training_args.hub_model_id
        repo = Repository(training_args.output_dir, clone_from=repo_name)

    # Get the datasets: you can either provide your own CSV/JSON/TXT training and evaluation files (see below)
    # or just provide the name of one of the public datasets for token classification task available on the hub at https://huggingface.co/datasets/
    # (the dataset will be downloaded automatically from the datasets Hub).
    #
    # For CSV/JSON files, this script will use the column called 'tokens' or the first column if no column called
    # 'tokens' is found. You can easily tweak this behavior (see below).
    #
    # In distributed training, the load_dataset function guarantee that only one local process can concurrently
    # download the dataset.
    if data_args.dataset_name is not None:
        # Downloading and loading a dataset from the hub.
        raw_datasets = load_dataset(
            data_args.dataset_name,
            data_args.dataset_config_name,
            cache_dir=model_args.cache_dir,
            use_auth_token=True if model_args.use_auth_token else None,
        )
    else:
        # Loading the dataset from local csv or json file.
        data_files = {}
        if data_args.train_file is not None:
            data_files["train"] = data_args.train_file
        if data_args.validation_file is not None:
            data_files["validation"] = data_args.validation_file
        if data_args.test_file is not None:
            data_files["test"] = data_args.test_file
        extension = (data_args.train_file if data_args.train_file is not None else data_args.valid_file).split(".")[-1]
        raw_datasets = load_dataset(
            extension,
            data_files=data_files,
            cache_dir=model_args.cache_dir,
            use_auth_token=True if model_args.use_auth_token else None,
        )
    # See more about loading any type of standard or custom dataset at
    # https://huggingface.co/docs/datasets/loading_datasets.html.

    if raw_datasets["train"] is not None:
        column_names = raw_datasets["train"].column_names
        features = raw_datasets["train"].features
    else:
        column_names = raw_datasets["validation"].column_names
        features = raw_datasets["validation"].features

    if data_args.text_column_name is not None:
        text_column_name = data_args.text_column_name
    elif "tokens" in column_names:
        text_column_name = "tokens"
    else:
        text_column_name = column_names[0]

    if data_args.label_column_name is not None:
        label_column_name = data_args.label_column_name
    elif f"{data_args.task_name}_tags" in column_names:
        label_column_name = f"{data_args.task_name}_tags"
    else:
        label_column_name = column_names[1]

    # In the event the labels are not a `Sequence[ClassLabel]`, we will need to go through the dataset to get the
    # unique labels.
    def get_label_list(labels):
        unique_labels = set()
        for label in labels:
            unique_labels = unique_labels | set(label)
        label_list = list(unique_labels)
        label_list.sort()
        return label_list

    if isinstance(features[label_column_name].feature, ClassLabel):
        label_list = features[label_column_name].feature.names
        # No need to convert the labels since they are already ints.
        label_to_id = {i: i for i in range(len(label_list))}
    else:
        label_list = get_label_list(raw_datasets["train"][label_column_name])
        label_to_id = {l: i for i, l in enumerate(label_list)}
    num_labels = len(label_list)

    # Load pretrained model and tokenizer
    config = AutoConfig.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        num_labels=num_labels,
        label2id=label_to_id,
        id2label={i: l for l, i in label_to_id.items()},
        finetuning_task=data_args.task_name,
        cache_dir=model_args.cache_dir,
        revision=model_args.model_revision,
        use_auth_token=True if model_args.use_auth_token else None,
    )
    tokenizer_name_or_path = model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path
    if config.model_type in {"gpt2", "roberta"}:
        tokenizer = AutoTokenizer.from_pretrained(
            tokenizer_name_or_path,
            cache_dir=model_args.cache_dir,
            revision=model_args.model_revision,
            use_auth_token=True if model_args.use_auth_token else None,
            add_prefix_space=True,
        )
    else:
        tokenizer = AutoTokenizer.from_pretrained(
            tokenizer_name_or_path,
            cache_dir=model_args.cache_dir,
            revision=model_args.model_revision,
            use_auth_token=True if model_args.use_auth_token else None,
        )
    model = FlaxAutoModelForTokenClassification.from_pretrained(
        model_args.model_name_or_path,
        config=config,
        from_pt=True,
        cache_dir=model_args.cache_dir,
        revision=model_args.model_revision,
        use_auth_token=True if model_args.use_auth_token else None,
    )

    # Preprocessing the datasets
    # Tokenize all texts and align the labels with them.
    def tokenize_and_align_labels(examples):
        tokenized_inputs = tokenizer(
            examples[text_column_name],
            max_length=data_args.max_seq_length,
            padding="max_length",
            truncation=True,
            # We use this argument because the texts in our dataset are lists of words (with a label for each word).
            is_split_into_words=True,
        )

        labels = []

        for i, label in enumerate(examples[label_column_name]):
            word_ids = tokenized_inputs.word_ids(batch_index=i)
            previous_word_idx = None
            label_ids = []
            for word_idx in word_ids:
                # Special tokens have a word id that is None. We set the label to -100 so they are automatically
                # ignored in the loss function.
                if word_idx is None:
                    label_ids.append(-100)
                # We set the label for the first token of each word.
                elif word_idx != previous_word_idx:
                    label_ids.append(label_to_id[label[word_idx]])
                # For the other tokens in a word, we set the label to either the current label or -100, depending on
                # the label_all_tokens flag.
                else:
                    label_ids.append(label_to_id[label[word_idx]] if data_args.label_all_tokens else -100)
                previous_word_idx = word_idx

            labels.append(label_ids)
        tokenized_inputs["labels"] = labels
        return tokenized_inputs

    processed_raw_datasets = raw_datasets.map(
        tokenize_and_align_labels,
        batched=True,
        num_proc=data_args.preprocessing_num_workers,
        load_from_cache_file=not data_args.overwrite_cache,
        remove_columns=raw_datasets["train"].column_names,
        desc="Running tokenizer on dataset",
    )

    train_dataset = processed_raw_datasets["train"]
    eval_dataset = processed_raw_datasets["validation"]
    test_dataset = processed_raw_datasets["test"]

    # Log a few random samples from the training set:
    for index in random.sample(range(len(train_dataset)), 3):
        logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

    # Define a summary writer
    has_tensorboard = is_tensorboard_available()
    if has_tensorboard and jax.process_index() == 0:
        try:
            from flax.metrics.tensorboard import SummaryWriter

            summary_writer = SummaryWriter(training_args.output_dir)
            summary_writer.hparams({**training_args.to_dict(), **vars(model_args), **vars(data_args)})
        except ImportError as ie:
            has_tensorboard = False
            logger.warning(
                f"Unable to display metrics through TensorBoard because some package are not installed: {ie}"
            )
    else:
        logger.warning(
            "Unable to display metrics through TensorBoard because the package is not installed: "
            "Please run pip install tensorboard to enable."
        )

    def write_train_metric(summary_writer, train_metrics, train_time, step):
        summary_writer.scalar("train_time", train_time, step)

        train_metrics = get_metrics(train_metrics)
        for key, vals in train_metrics.items():
            tag = f"train_{key}"
            for i, val in enumerate(vals):
                summary_writer.scalar(tag, val, step - len(vals) + i + 1)

    def write_eval_metric(summary_writer, eval_metrics, step):
        for metric_name, value in eval_metrics.items():
            summary_writer.scalar(f"eval_{metric_name}", value, step)

    num_epochs = int(training_args.num_train_epochs)
    rng = jax.random.PRNGKey(training_args.seed)
    dropout_rngs = jax.random.split(rng, jax.local_device_count())

    train_batch_size = training_args.per_device_train_batch_size * jax.local_device_count()
    per_device_eval_batch_size = int(training_args.per_device_eval_batch_size)
    eval_batch_size = training_args.per_device_eval_batch_size * jax.local_device_count()

    learning_rate_fn = create_learning_rate_fn(
        len(train_dataset),
        train_batch_size,
        training_args.num_train_epochs,
        training_args.warmup_steps,
        training_args.learning_rate,
    )

    state = create_train_state(model, learning_rate_fn, num_labels=num_labels, training_args=training_args)

    # define step functions
    def train_step(
        state: train_state.TrainState, batch: Dict[str, Array], dropout_rng: PRNGKey
    ) -> Tuple[train_state.TrainState, float]:
        """Trains model with an optimizer (both in `state`) on `batch`, returning a pair `(new_state, loss)`."""
        dropout_rng, new_dropout_rng = jax.random.split(dropout_rng)
        targets = batch.pop("labels")

        def loss_fn(params):
            logits = state.apply_fn(**batch, params=params, dropout_rng=dropout_rng, train=True)[0]
            loss = state.loss_fn(logits, targets)
            return loss

        grad_fn = jax.value_and_grad(loss_fn)
        loss, grad = grad_fn(state.params)
        grad = jax.lax.pmean(grad, "batch")
        new_state = state.apply_gradients(grads=grad)
        metrics = jax.lax.pmean({"loss": loss, "learning_rate": learning_rate_fn(state.step)}, axis_name="batch")
        return new_state, metrics, new_dropout_rng

    p_train_step = jax.pmap(train_step, axis_name="batch", donate_argnums=(0,))

    def eval_step(state, batch):
        logits = state.apply_fn(**batch, params=state.params, train=False)[0]
        return state.logits_fn(logits)

    p_eval_step = jax.pmap(eval_step, axis_name="batch")

    metric = evaluate.load("seqeval")

    def get_labels(y_pred, y_true):
        # Transform predictions and references tensos to numpy arrays

        # Remove ignored index (special tokens)
        true_predictions = [
            [label_list[p] for (p, l) in zip(pred, gold_label) if l != -100]
            for pred, gold_label in zip(y_pred, y_true)
        ]
        true_labels = [
            [label_list[l] for (p, l) in zip(pred, gold_label) if l != -100]
            for pred, gold_label in zip(y_pred, y_true)
        ]
        return true_predictions, true_labels

    def compute_metrics():
        results = metric.compute()
        if data_args.return_entity_level_metrics:
            # Unpack nested dictionaries
            final_results = {}
            for key, value in results.items():
                if isinstance(value, dict):
                    for n, v in value.items():
                        final_results[f"{key}_{n}"] = v
                else:
                    final_results[key] = value
            return final_results
        if data_args.return_macro_metrics:
            Ps, Rs, Fs = [], [], []
            for type_name in results:
                if type_name.startswith("overall"):
                    continue
                Ps.append(results[type_name]["precision"])
                Rs.append(results[type_name]["recall"])
                Fs.append(results[type_name]["f1"])
            return {
                "macro_precision": np.mean(Ps),
                "macro_recall": np.mean(Rs),
                "macro_f1": np.mean(Fs),
            }
        else:
            return {
                "precision": results["overall_precision"],
                "recall": results["overall_recall"],
                "f1": results["overall_f1"],
                "accuracy": results["overall_accuracy"],
            }

    logger.info(f"===== Starting training ({num_epochs} epochs) =====")
    train_time = 0

    # make sure weights are replicated on each device
    state = replicate(state)

    train_time = 0
    step_per_epoch = len(train_dataset) // train_batch_size
    total_steps = step_per_epoch * num_epochs
    epochs = tqdm(range(num_epochs), desc=f"Epoch ... (1/{num_epochs})", position=0, leave=True)
    for epoch in epochs:

        train_start = time.time()
        train_metrics = []

        # Create sampling rng
        rng, input_rng = jax.random.split(rng)

        # train
        for step, batch in enumerate(
            tqdm(
                train_data_collator(input_rng, train_dataset, train_batch_size),
                total=step_per_epoch,
                desc="Training...",
                position=0, leave=True,
            )
        ):
            state, train_metric, dropout_rngs = p_train_step(state, batch, dropout_rngs)
            train_metrics.append(train_metric)

            cur_step = (epoch * step_per_epoch) + (step + 1)

            if cur_step % training_args.logging_steps == 0 and cur_step > 0:
                # Save metrics
                train_metric = unreplicate(train_metric)
                train_time += time.time() - train_start
                if has_tensorboard and jax.process_index() == 0:
                    write_train_metric(summary_writer, train_metrics, train_time, cur_step)

                epochs.write(
                    f"Step... ({cur_step}/{total_steps} | Training Loss: {train_metric['loss']}, Learning Rate:"
                    f" {train_metric['learning_rate']})"
                )

                train_metrics = []

            if (cur_step % training_args.eval_steps == 0 or cur_step % step_per_epoch == 0) and cur_step > 0:

                eval_metrics = {}
                # evaluate
                for batch in tqdm(
                    eval_data_collator(eval_dataset, eval_batch_size),
                    total=math.ceil(len(eval_dataset) / eval_batch_size),
                    desc="Evaluating on Dev Set...",
                    position=0, leave=True,
                ):
                    labels = batch.pop("labels")
                    predictions = pad_shard_unpad(p_eval_step)(
                        state, batch, min_device_batch=per_device_eval_batch_size
                    )
                    predictions = np.array(predictions)
                    labels[np.array(chain(*batch["attention_mask"])) == 0] = -100
                    preds, refs = get_labels(predictions, labels)
                    metric.add_batch(
                        predictions=preds,
                        references=refs,
                    )

                eval_metrics = compute_metrics()
                wandb.log(eval_metrics)
                logger.info(f"Step... ({cur_step}/{total_steps} | Validation metrics: {eval_metrics}")

                if has_tensorboard and jax.process_index() == 0:
                    write_eval_metric(summary_writer, eval_metrics, cur_step)

            #if (cur_step % training_args.save_steps == 0 and cur_step > 0) or (cur_step == total_steps):
            if 10==11:
                # save checkpoint after each epoch and push checkpoint to the hub
                if jax.process_index() == 0:
                    params = jax.device_get(unreplicate(state.params))
                    model.save_pretrained(training_args.output_dir, params=params)
                    tokenizer.save_pretrained(training_args.output_dir)
                    if training_args.push_to_hub:
                        repo.push_to_hub(commit_message=f"Saving weights and logs of step {cur_step}", blocking=False)
        epochs.desc = f"Epoch ... {epoch + 1}/{num_epochs}"

    # Eval after training
    if training_args.do_eval:
        eval_metrics = {}
        eval_loader = eval_data_collator(eval_dataset, eval_batch_size)
        for batch in tqdm(eval_loader, total=len(eval_dataset) // eval_batch_size, desc="Evaluating on Dev Set ...", position=0, leave=True):
            labels = batch.pop("labels")
            predictions = pad_shard_unpad(p_eval_step)(state, batch, min_device_batch=per_device_eval_batch_size)
            predictions = np.array(predictions)
            labels[np.array(chain(*batch["attention_mask"])) == 0] = -100
            preds, refs = get_labels(predictions, labels)
            metric.add_batch(predictions=preds, references=refs)

        eval_metrics = compute_metrics()
        logger.info(f"Evaluation Results on Dev Set : {eval_metrics}")
    if training_args.do_test:
        eval_metrics = {}
        eval_loader = eval_data_collator(test_dataset, eval_batch_size)
        preds_list=[]
        preds_dict={}
        labels_list=[]
        for batch in tqdm(eval_loader, total=len(test_dataset) // eval_batch_size, desc="Evaluating on Test Set...", position=0, leave=True):
            labels = batch.pop("labels")
            predictions = pad_shard_unpad(p_eval_step)(state, batch, min_device_batch=per_device_eval_batch_size)
            predictions = np.array(predictions)
            labels[np.array(chain(*batch["attention_mask"])) == 0] = -100
            preds, refs = get_labels(predictions, labels)
            for pred_example in preds:
                      preds_list.append(pred_example)
            for label_example in refs:
                      labels_list.append(label_example)
            metric.add_batch(predictions=preds, references=refs)
        raw_ids=[]
        for row in raw_datasets["test"]:
          raw_ids.append(row["id"])
        for label_id,pred_value in zip(raw_ids,preds_list):
              preds_dict[label_id]=pred_value
        if "/" in model_args.model_name_or_path:
                   model_name_hf=model_args.model_name_or_path.split("/")[-1]
        else:
                   model_name_hf=model_args.model_name_or_path
        final_entry={"dataset_name": data_args.blurb_task, "model_name":model_name_hf, "predictions":preds_dict}
        with open(mmodel_name_hf+"_"+str(data_args.blurb_task)+".json", "w") as f:
          json.dump(final_entry, f, indent=4, sort_keys=True)
        eval_metrics = compute_metrics()
        logger.info(f"Evaluation Results on Test Set : {eval_metrics}")
        #if jax.process_index() == 0:
        #    eval_metrics = {f"eval_{metric_name}": value for metric_name, value in eval_metrics.items()}
        #    path = os.path.join(training_args.output_dir, "eval_results.json")
        #    with open(path, "w") as f:
        #        json.dump(eval_metrics, f, indent=4, sort_keys=True)


if __name__ == "__main__":
    main()


Writing run_flax_ner2.py


Please note that our code will show both eval and test results and also will output the prediction as BLURB format as [model-name-task.json] (e.g BioM-ALBERT-xxlarge-PMC_BC5-disease.json).


# **BioM-ALBERT**

In [ ]:
!python3 run_flax_seq.py --model_name_or_path sultan/BioM-ALBERT-xxlarge \
--train_file data/seqcls/chemprot_hf/train.json \
--validation_file data/seqcls/chemprot_hf/dev.json \
--test_file data/seqcls/chemprot_hf/test.json \
--do_train \
--do_eval \
--per_device_train_batch_size 3 \
--per_device_eval_batch_size 8 \
--learning_rate 3e-5 \
--num_train_epochs 6 \
--max_seq_length 256 \
--save_strategy no \
--evaluation_strategy no \
--output_dir out \
--overwrite_output_dir \
--save_strategy no \
--evaluation_strategy no \
--eval_steps 100000 \
--metric_name PRF1 \
--blurb_task ChemProt

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 647.97it/s]
['0', 'CPR:3', 'CPR:4', 'CPR:5', 'CPR:6', 'CPR:9']
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ALBERT-xxlarge/snapshots/c85649a7b3345b7de438e59c03f8f702e3aebc76/config.json
Model config AlbertConfig {
  "_name_or_path": "sultan/BioM-ALBERT-xxlarge",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 4096,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.01,
  "inner_group_num": 1,
  "intermediate_size": 16384,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
 

In [ ]:
!python3 run_flax_seq.py --model_name_or_path sultan/BioM-ALBERT-xxlarge  \
--train_file data/seqcls/DDI_hf/train.json \
--validation_file data/seqcls/DDI_hf/dev.json \
--test_file data/seqcls/DDI_hf/test.json \
--do_train \
--do_eval \
--per_device_train_batch_size 1 \
--per_device_eval_batch_size 8 \
--learning_rate 2e-5 \
--num_train_epochs 3 \
--max_seq_length 256 \
--save_strategy no \
--evaluation_strategy no \
--output_dir out \
--overwrite_output_dir \
--save_strategy no \
--evaluation_strategy no \
--eval_steps 100000 \
--metric_name PRF1 \
--blurb_task DDI

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 673.89it/s]
['0', 'DDI-advise', 'DDI-effect', 'DDI-int', 'DDI-mechanism']
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ALBERT-xxlarge/snapshots/c85649a7b3345b7de438e59c03f8f702e3aebc76/config.json
Model config AlbertConfig {
  "_name_or_path": "sultan/BioM-ALBERT-xxlarge",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 4096,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.01,
  "inner_group_num": 1,
  "intermediate_size": 16384,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL

In [ ]:
!python3 run_flax_seq.py --model_name_or_path sultan/BioM-ALBERT-xxlarge  \
--train_file data/seqcls/GAD_hf/train.json \
--validation_file data/seqcls/GAD_hf/dev.json \
--test_file data/seqcls/GAD_hf/test.json \
--do_train \
--do_eval \
--per_device_train_batch_size 3 \
--per_device_eval_batch_size 8 \
--learning_rate 2e-5 \
--num_train_epochs 8 \
--max_seq_length 256 \
--save_strategy no \
--evaluation_strategy no \
--output_dir out \
--overwrite_output_dir \
--save_strategy no \
--evaluation_strategy no \
--eval_steps 100000 \
--metric_name PRF1 \
--blurb_task GAD

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 854.12it/s]
['0', '1']
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ALBERT-xxlarge/snapshots/c85649a7b3345b7de438e59c03f8f702e3aebc76/config.json
Model config AlbertConfig {
  "_name_or_path": "sultan/BioM-ALBERT-xxlarge",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 4096,
  "initializer_range": 0.01,
  "inner_group_num": 1,
  "intermediate_size": 16384,
  "layer_norm_eps": 1e-12,
  "layers_to_keep": [],
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 64,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pa

In [ ]:
!python3 run_flax_seq.py --model_name_or_path sultan/BioM-ALBERT-xxlarge  \
--train_file data/seqcls/bioasq_hf/train.json \
--validation_file data/seqcls/bioasq_hf/dev.json \
--test_file data/seqcls/bioasq_hf/test.json \
--do_train \
--do_eval \
--per_device_train_batch_size 1 \
--per_device_eval_batch_size 8 \
--learning_rate 3e-5 \
--num_train_epochs 5 \
--max_seq_length 512 \
--save_strategy no \
--evaluation_strategy no \
--output_dir out \
--overwrite_output_dir \
--save_strategy no \
--evaluation_strategy no \
--eval_steps 100000 \
--blurb_task BioASQ

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 809.19it/s]
['no', 'yes']
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ALBERT-xxlarge/snapshots/c85649a7b3345b7de438e59c03f8f702e3aebc76/config.json
Model config AlbertConfig {
  "_name_or_path": "sultan/BioM-ALBERT-xxlarge",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 4096,
  "initializer_range": 0.01,
  "inner_group_num": 1,
  "intermediate_size": 16384,
  "layer_norm_eps": 1e-12,
  "layers_to_keep": [],
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 64,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  

In [ ]:
!python3 run_flax_seq.py --model_name_or_path sultan/BioM-ALBERT-xxlarge \
--train_file data/seqcls/pubmedqa_hf/train.json \
--validation_file data/seqcls/pubmedqa_hf/dev.json \
--test_file data/seqcls/pubmedqa_hf/test.json \
--do_train \
--do_eval \
--per_device_train_batch_size 1 \
--per_device_eval_batch_size 8 \
--learning_rate 2e-5 \
--num_train_epochs 7 \
--max_seq_length 512 \
--save_strategy no \
--evaluation_strategy no \
--output_dir out \
--overwrite_output_dir \
--save_strategy no \
--evaluation_strategy no \
--eval_steps 100000 \
--blurb_task PubMedQA

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 839.08it/s]
['maybe', 'no', 'yes']
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ALBERT-xxlarge/snapshots/c85649a7b3345b7de438e59c03f8f702e3aebc76/config.json
Model config AlbertConfig {
  "_name_or_path": "sultan/BioM-ALBERT-xxlarge",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 4096,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.01,
  "inner_group_num": 1,
  "intermediate_size": 16384,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "layers_to_keep": [],
  "max_position_embeddings": 512,


In [ ]:
!python3 run_flax_seq.py --model_name_or_path sultan/BioM-ALBERT-xxlarge \
--train_file data/seqcls/BIOSSES_hf/train.json \
--validation_file data/seqcls/BIOSSES_hf/dev.json \
--test_file data/seqcls/BIOSSES_hf/test.json \
--do_train \
--do_eval \
--per_device_train_batch_size 1 \
--per_device_eval_batch_size 8 \
--learning_rate 5e-5 \
--num_train_epochs 70 \
--max_seq_length 512 \
--save_strategy no \
--evaluation_strategy no \
--output_dir out \
--overwrite_output_dir \
--save_strategy no \
--evaluation_strategy no \
--eval_steps 100000 \
--metric_name pearsonr \
--blurb_task BIOSSES

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 849.28it/s]
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ALBERT-xxlarge/snapshots/c85649a7b3345b7de438e59c03f8f702e3aebc76/config.json
Model config AlbertConfig {
  "_name_or_path": "sultan/BioM-ALBERT-xxlarge",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 4096,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.01,
  "inner_group_num": 1,
  "intermediate_size": 16384,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "layers_to_keep": [],
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 64,
  "num_hidden_gro

In [ ]:
!python3 run_flax_ner.py --model_name_or_path sultan/BioM-ALBERT-xxlarge \
--train_file data/tokcls/BC5CDR-chem_hf/train.json \
--validation_file data/tokcls/BC5CDR-chem_hf/dev.json \
--test_file data/tokcls/BC5CDR-chem_hf/test.json \
--do_train \
--do_eval \
--do_test \
--per_device_train_batch_size 3 \
--learning_rate 5e-5 \
--num_train_epochs 6 \
--output_dir out \
--overwrite_output_dir \
--blurb_task BC5CDR-chem \
--max_seq_length 512 \
--return_entity_level_metrics \
--eval_steps 10000000

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 754.82it/s]
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ALBERT-xxlarge/snapshots/c85649a7b3345b7de438e59c03f8f702e3aebc76/config.json
Model config AlbertConfig {
  "_name_or_path": "sultan/BioM-ALBERT-xxlarge",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "finetuning_task": "ner",
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 4096,
  "id2label": {
    "0": "B",
    "1": "I",
    "2": "O"
  },
  "initializer_range": 0.01,
  "inner_group_num": 1,
  "intermediate_size": 16384,
  "label2id": {
    "B": 0,
    "I": 1,
    "O": 2
  },
  "layer_norm_eps": 1e-12,
  "layers_to_keep": [],
  "max_position_embeddings": 512,
  "model_type": "albert",
  "ne

In [ ]:
!python3 run_flax_ner.py --model_name_or_path sultan/BioM-ALBERT-xxlarge \
--train_file data/tokcls/BC5CDR-disease_hf/train.json \
--validation_file data/tokcls/BC5CDR-disease_hf/dev.json \
--test_file data/tokcls/BC5CDR-disease_hf/test.json \
--do_train \
--do_eval \
--do_test \
--per_device_train_batch_size 1 \
--learning_rate 4e-5 \
--num_train_epochs 5 \
--output_dir out \
--overwrite_output_dir \
--blurb_task BC5-disease \
--max_seq_length 512 \
--return_entity_level_metrics \
--eval_steps 1000

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 776.00it/s]
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ALBERT-xxlarge/snapshots/c85649a7b3345b7de438e59c03f8f702e3aebc76/config.json
Model config AlbertConfig {
  "_name_or_path": "sultan/BioM-ALBERT-xxlarge",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "finetuning_task": "ner",
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 4096,
  "id2label": {
    "0": "B",
    "1": "I",
    "2": "O"
  },
  "initializer_range": 0.01,
  "inner_group_num": 1,
  "intermediate_size": 16384,
  "label2id": {
    "B": 0,
    "I": 1,
    "O": 2
  },
  "layer_norm_eps": 1e-12,
  "layers_to_keep": [],
  "max_position_embeddings": 512,
  "model_type": "albert",
  "ne

In [ ]:
!python3 run_flax_ner.py --model_name_or_path sultan/BioM-ALBERT-xxlarge \
--train_file data/tokcls/BC2GM_hf/train.json \
--validation_file data/tokcls/BC2GM_hf/dev.json \
--test_file data/tokcls/BC2GM_hf/test.json \
--do_train \
--do_eval \
--do_test \
--per_device_train_batch_size 3 \
--learning_rate 7e-5 \
--num_train_epochs 9 \
--output_dir out \
--overwrite_output_dir \
--blurb_task BC2GM \
--max_seq_length 512 \
--return_entity_level_metrics \
--eval_steps 1000

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 712.51it/s]
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ALBERT-xxlarge/snapshots/c85649a7b3345b7de438e59c03f8f702e3aebc76/config.json
Model config AlbertConfig {
  "_name_or_path": "sultan/BioM-ALBERT-xxlarge",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "finetuning_task": "ner",
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 4096,
  "id2label": {
    "0": "B",
    "1": "I",
    "2": "O"
  },
  "initializer_range": 0.01,
  "inner_group_num": 1,
  "intermediate_size": 16384,
  "label2id": {
    "B": 0,
    "I": 1,
    "O": 2
  },
  "layer_norm_eps": 1e-12,
  "layers_to_keep": [],
  "max_position_embeddings": 512,
  "model_type": "albert",
  "ne

In [ ]:
!python3 run_flax_ner.py --model_name_or_path sultan/BioM-ALBERT-xxlarge \
--train_file data/tokcls/NCBI-disease_hf/train.json \
--validation_file data/tokcls/NCBI-disease_hf/dev.json \
--test_file data/tokcls/NCBI-disease_hf/test.json \
--do_train \
--do_eval \
--do_test \
--per_device_train_batch_size 3 \
--learning_rate 7e-5 \
--num_train_epochs 10 \
--output_dir out \
--overwrite_output_dir \
--blurb_task NCBI-disease \
--max_seq_length 512 \
--return_entity_level_metrics \
--eval_steps 1000

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 753.56it/s]
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ALBERT-xxlarge/snapshots/c85649a7b3345b7de438e59c03f8f702e3aebc76/config.json
Model config AlbertConfig {
  "_name_or_path": "sultan/BioM-ALBERT-xxlarge",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "finetuning_task": "ner",
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 4096,
  "id2label": {
    "0": "B",
    "1": "I",
    "2": "O"
  },
  "initializer_range": 0.01,
  "inner_group_num": 1,
  "intermediate_size": 16384,
  "label2id": {
    "B": 0,
    "I": 1,
    "O": 2
  },
  "layer_norm_eps": 1e-12,
  "layers_to_keep": [],
  "max_position_embeddings": 512,
  "model_type": "albert",
  "ne

In [ ]:
!python3 run_flax_ner.py --model_name_or_path sultan/BioM-ALBERT-xxlarge \
--train_file data/tokcls/JNLPBA_hf/train.json \
--validation_file data/tokcls/JNLPBA_hf/dev.json \
--test_file data/tokcls/JNLPBA_hf/test.json \
--do_train \
--do_eval \
--do_test \
--per_device_train_batch_size 3 \
--learning_rate 3e-5 \
--num_train_epochs 20 \
--output_dir out \
--overwrite_output_dir \
--blurb_task JNLPBA \
--max_seq_length 512 \
--return_entity_level_metrics \
--eval_steps 1000

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 789.19it/s]
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ALBERT-xxlarge/snapshots/c85649a7b3345b7de438e59c03f8f702e3aebc76/config.json
Model config AlbertConfig {
  "_name_or_path": "sultan/BioM-ALBERT-xxlarge",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "finetuning_task": "ner",
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 4096,
  "id2label": {
    "0": "B",
    "1": "I",
    "2": "O"
  },
  "initializer_range": 0.01,
  "inner_group_num": 1,
  "intermediate_size": 16384,
  "label2id": {
    "B": 0,
    "I": 1,
    "O": 2
  },
  "layer_norm_eps": 1e-12,
  "layers_to_keep": [],
  "max_position_embeddings": 512,
  "model_type": "albert",
  "ne

In [ ]:
!python3 run_flax_ner.py --model_name_or_path sultan/BioM-ALBERT-xxlarge \
--train_file data/tokcls/ebmnlp_hf/train.json \
--validation_file data/tokcls/ebmnlp_hf/dev.json \
--test_file data/tokcls/ebmnlp_hf/test.json \
--do_train \
--do_eval \
--do_test \
--per_device_train_batch_size 2 \
--learning_rate 2e-5 \
--num_train_epochs 1 \
--output_dir out \
--overwrite_output_dir \
--blurb_task "EBM PICO" \
--max_seq_length 512 \
--return_macro_metrics \
--eval_steps 1000

100%|█████████████████████████████████████████████| 3/3 [00:00<00:00,  5.82it/s]
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ALBERT-xxlarge/snapshots/c85649a7b3345b7de438e59c03f8f702e3aebc76/config.json
Model config AlbertConfig {
  "_name_or_path": "sultan/BioM-ALBERT-xxlarge",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "finetuning_task": "ner",
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 4096,
  "id2label": {
    "0": "B-INT",
    "1": "B-OUT",
    "2": "B-PAR",
    "3": "O"
  },
  "initializer_range": 0.01,
  "inner_group_num": 1,
  "intermediate_size": 16384,
  "label2id": {
    "B-INT": 0,
    "B-OUT": 1,
    "B-PAR": 2,
    "O": 3
  },
  "layer_norm_eps": 1e-12,
  "layers_to_keep": [],
  "max_position

# **BioM-ALBERT-PMC**

In [ ]:
!python3 run_flax_seq.py --model_name_or_path sultan/BioM-ALBERT-xxlarge-PMC \
--train_file data/seqcls/chemprot_hf/train.json \
--validation_file data/seqcls/chemprot_hf/dev.json \
--test_file data/seqcls/chemprot_hf/test.json \
--do_train \
--do_eval \
--per_device_train_batch_size 1 \
--per_device_eval_batch_size 8 \
--learning_rate 2e-5 \
--num_train_epochs 5 \
--max_seq_length 256 \
--save_strategy no \
--evaluation_strategy no \
--output_dir out \
--overwrite_output_dir \
--save_strategy no \
--evaluation_strategy no \
--eval_steps 100000 \
--metric_name PRF1 \
--blurb_task ChemProt

100%|█████████████████████████████████████████████| 3/3 [00:00<00:00,  4.24it/s]
['0', 'CPR:3', 'CPR:4', 'CPR:5', 'CPR:6', 'CPR:9']
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ALBERT-xxlarge-PMC/snapshots/047499f199be4e57c5dd131a355914131d9c9669/config.json
Model config AlbertConfig {
  "_name_or_path": "sultan/BioM-ALBERT-xxlarge-PMC",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 4096,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.01,
  "inner_group_num": 1,
  "intermediate_size": 16384,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_

In [ ]:
!python3 run_flax_seq.py --model_name_or_path sultan/BioM-ALBERT-xxlarge-PMC  \
--train_file data/seqcls/DDI_hf/train.json \
--validation_file data/seqcls/DDI_hf/dev.json \
--test_file data/seqcls/DDI_hf/test.json \
--do_train \
--do_eval \
--per_device_train_batch_size 3 \
--per_device_eval_batch_size 8 \
--learning_rate 4e-5 \
--num_train_epochs 5 \
--max_seq_length 256 \
--save_strategy no \
--evaluation_strategy no \
--output_dir out \
--overwrite_output_dir \
--save_strategy no \
--evaluation_strategy no \
--eval_steps 100000 \
--metric_name PRF1 \
--blurb_task DDI

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 738.09it/s]
['0', 'DDI-advise', 'DDI-effect', 'DDI-int', 'DDI-mechanism']
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ALBERT-xxlarge-PMC/snapshots/047499f199be4e57c5dd131a355914131d9c9669/config.json
Model config AlbertConfig {
  "_name_or_path": "sultan/BioM-ALBERT-xxlarge-PMC",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 4096,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.01,
  "inner_group_num": 1,
  "intermediate_size": 16384,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
  

In [ ]:
!python3 run_flax_seq.py --model_name_or_path sultan/BioM-ALBERT-xxlarge-PMC \
--train_file data/seqcls/GAD_hf/train.json \
--validation_file data/seqcls/GAD_hf/dev.json \
--test_file data/seqcls/GAD_hf/test.json \
--do_train \
--do_eval \
--per_device_train_batch_size 3 \
--per_device_eval_batch_size 8 \
--learning_rate 4e-5 \
--num_train_epochs 8 \
--max_seq_length 256 \
--save_strategy no \
--evaluation_strategy no \
--output_dir out \
--overwrite_output_dir \
--save_strategy no \
--evaluation_strategy no \
--eval_steps 100000 \
--metric_name PRF1 \
--blurb_task GAD

100%|█████████████████████████████████████████████| 3/3 [00:00<00:00, 48.40it/s]
['0', '1']
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ALBERT-xxlarge-PMC/snapshots/047499f199be4e57c5dd131a355914131d9c9669/config.json
Model config AlbertConfig {
  "_name_or_path": "sultan/BioM-ALBERT-xxlarge-PMC",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 4096,
  "initializer_range": 0.01,
  "inner_group_num": 1,
  "intermediate_size": 16384,
  "layer_norm_eps": 1e-12,
  "layers_to_keep": [],
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 64,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 

In [ ]:
!python3 run_flax_seq.py --model_name_or_path sultan/BioM-ALBERT-xxlarge-PMC  \
--train_file data/seqcls/bioasq_hf/train.json \
--validation_file data/seqcls/bioasq_hf/dev.json \
--test_file data/seqcls/bioasq_hf/test.json \
--do_train \
--do_eval \
--per_device_train_batch_size 1 \
--per_device_eval_batch_size 8 \
--learning_rate 2e-5 \
--num_train_epochs 20 \
--max_seq_length 512 \
--save_strategy no \
--evaluation_strategy no \
--output_dir out \
--overwrite_output_dir \
--save_strategy no \
--evaluation_strategy no \
--eval_steps 100000 \
--blurb_task BioASQ

100%|█████████████████████████████████████████████| 3/3 [00:00<00:00, 25.30it/s]
['no', 'yes']
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ALBERT-xxlarge-PMC/snapshots/047499f199be4e57c5dd131a355914131d9c9669/config.json
Model config AlbertConfig {
  "_name_or_path": "sultan/BioM-ALBERT-xxlarge-PMC",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 4096,
  "initializer_range": 0.01,
  "inner_group_num": 1,
  "intermediate_size": 16384,
  "layer_norm_eps": 1e-12,
  "layers_to_keep": [],
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 64,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks

In [ ]:
!python3 run_flax_seq.py --model_name_or_path sultan/BioM-ALBERT-xxlarge-PMC \
--train_file data/seqcls/pubmedqa_hf/train.json \
--validation_file data/seqcls/pubmedqa_hf/dev.json \
--test_file data/seqcls/pubmedqa_hf/test.json \
--do_train \
--do_eval \
--per_device_train_batch_size 1 \
--per_device_eval_batch_size 8 \
--learning_rate 3e-5 \
--num_train_epochs 8 \
--max_seq_length 512 \
--save_strategy no \
--evaluation_strategy no \
--output_dir out \
--overwrite_output_dir \
--save_strategy no \
--evaluation_strategy no \
--eval_steps 100000 \
--blurb_task PubMedQA

100%|█████████████████████████████████████████████| 3/3 [00:00<00:00, 35.55it/s]
['maybe', 'no', 'yes']
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ALBERT-xxlarge-PMC/snapshots/047499f199be4e57c5dd131a355914131d9c9669/config.json
Model config AlbertConfig {
  "_name_or_path": "sultan/BioM-ALBERT-xxlarge-PMC",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 4096,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.01,
  "inner_group_num": 1,
  "intermediate_size": 16384,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "layers_to_keep": [],
  "max_position_embeddings

In [ ]:
!python3 run_flax_seq.py --model_name_or_path sultan/BioM-ALBERT-xxlarge-PMC \
--train_file data/seqcls/BIOSSES_hf/train.json \
--validation_file data/seqcls/BIOSSES_hf/dev.json \
--test_file data/seqcls/BIOSSES_hf/test.json \
--do_train \
--do_eval \
--per_device_train_batch_size 1 \
--per_device_eval_batch_size 8 \
--learning_rate 5e-5 \
--num_train_epochs 60 \
--max_seq_length 512 \
--save_strategy no \
--evaluation_strategy no \
--output_dir out \
--overwrite_output_dir \
--save_strategy no \
--evaluation_strategy no \
--eval_steps 100000 \
--metric_name pearsonr \
--blurb_task BIOSSES

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 670.30it/s]
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ALBERT-xxlarge-PMC/snapshots/047499f199be4e57c5dd131a355914131d9c9669/config.json
Model config AlbertConfig {
  "_name_or_path": "sultan/BioM-ALBERT-xxlarge-PMC",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 4096,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.01,
  "inner_group_num": 1,
  "intermediate_size": 16384,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "layers_to_keep": [],
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 64,
  "num_hi

In [ ]:
!python3 run_flax_ner.py --model_name_or_path sultan/BioM-ALBERT-xxlarge-PMC \
--train_file data/tokcls/BC5CDR-chem_hf/train.json \
--validation_file data/tokcls/BC5CDR-chem_hf/dev.json \
--test_file data/tokcls/BC5CDR-chem_hf/test.json \
--do_train \
--do_eval \
--do_test \
--per_device_train_batch_size 3 \
--learning_rate 5e-5 \
--num_train_epochs 6 \
--output_dir out \
--overwrite_output_dir \
--blurb_task BC5CDR-chem \
--max_seq_length 512 \
--return_entity_level_metrics \
--eval_steps 10000000

100%|█████████████████████████████████████████████| 3/3 [00:00<00:00, 13.07it/s]
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ALBERT-xxlarge-PMC/snapshots/047499f199be4e57c5dd131a355914131d9c9669/config.json
Model config AlbertConfig {
  "_name_or_path": "sultan/BioM-ALBERT-xxlarge-PMC",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "finetuning_task": "ner",
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 4096,
  "id2label": {
    "0": "B",
    "1": "I",
    "2": "O"
  },
  "initializer_range": 0.01,
  "inner_group_num": 1,
  "intermediate_size": 16384,
  "label2id": {
    "B": 0,
    "I": 1,
    "O": 2
  },
  "layer_norm_eps": 1e-12,
  "layers_to_keep": [],
  "max_position_embeddings": 512,
  "model_type": "albert

In [ ]:
!python3 run_flax_ner.py --model_name_or_path sultan/BioM-ALBERT-xxlarge-PMC \
--train_file data/tokcls/BC5CDR-disease_hf/train.json \
--validation_file data/tokcls/BC5CDR-disease_hf/dev.json \
--test_file data/tokcls/BC5CDR-disease_hf/test.json \
--do_train \
--do_eval \
--do_test \
--per_device_train_batch_size 1 \
--learning_rate 8e-5 \
--num_train_epochs 5 \
--output_dir out \
--overwrite_output_dir \
--blurb_task BC5-disease \
--max_seq_length 512 \
--return_entity_level_metrics \
--eval_steps 1000

100%|█████████████████████████████████████████████| 3/3 [00:00<00:00, 24.10it/s]
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ALBERT-xxlarge-PMC/snapshots/047499f199be4e57c5dd131a355914131d9c9669/config.json
Model config AlbertConfig {
  "_name_or_path": "sultan/BioM-ALBERT-xxlarge-PMC",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "finetuning_task": "ner",
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 4096,
  "id2label": {
    "0": "B",
    "1": "I",
    "2": "O"
  },
  "initializer_range": 0.01,
  "inner_group_num": 1,
  "intermediate_size": 16384,
  "label2id": {
    "B": 0,
    "I": 1,
    "O": 2
  },
  "layer_norm_eps": 1e-12,
  "layers_to_keep": [],
  "max_position_embeddings": 512,
  "model_type": "albert

In [ ]:
!python3 run_flax_ner.py --model_name_or_path sultan/BioM-ALBERT-xxlarge-PMC \
--train_file data/tokcls/BC2GM_hf/train.json \
--validation_file data/tokcls/BC2GM_hf/dev.json \
--test_file data/tokcls/BC2GM_hf/test.json \
--do_train \
--do_eval \
--do_test \
--per_device_train_batch_size 2 \
--learning_rate 5e-5 \
--num_train_epochs 6 \
--output_dir out \
--overwrite_output_dir \
--blurb_task BC2GM \
--max_seq_length 512 \
--return_entity_level_metrics \
--eval_steps 1000

100%|█████████████████████████████████████████████| 3/3 [00:00<00:00,  8.14it/s]
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ALBERT-xxlarge-PMC/snapshots/047499f199be4e57c5dd131a355914131d9c9669/config.json
Model config AlbertConfig {
  "_name_or_path": "sultan/BioM-ALBERT-xxlarge-PMC",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "finetuning_task": "ner",
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 4096,
  "id2label": {
    "0": "B",
    "1": "I",
    "2": "O"
  },
  "initializer_range": 0.01,
  "inner_group_num": 1,
  "intermediate_size": 16384,
  "label2id": {
    "B": 0,
    "I": 1,
    "O": 2
  },
  "layer_norm_eps": 1e-12,
  "layers_to_keep": [],
  "max_position_embeddings": 512,
  "model_type": "albert

In [ ]:
!python3 run_flax_ner.py --model_name_or_path sultan/BioM-ALBERT-xxlarge-PMC \
--train_file data/tokcls/NCBI-disease_hf/train.json \
--validation_file data/tokcls/NCBI-disease_hf/dev.json \
--test_file data/tokcls/NCBI-disease_hf/test.json \
--do_train \
--do_eval \
--do_test \
--per_device_train_batch_size 2 \
--learning_rate 7e-5 \
--num_train_epochs 5 \
--output_dir out \
--overwrite_output_dir \
--blurb_task NCBI-disease \
--max_seq_length 512 \
--return_entity_level_metrics \
--eval_steps 1000

100%|█████████████████████████████████████████████| 3/3 [00:00<00:00, 26.69it/s]
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ALBERT-xxlarge-PMC/snapshots/047499f199be4e57c5dd131a355914131d9c9669/config.json
Model config AlbertConfig {
  "_name_or_path": "sultan/BioM-ALBERT-xxlarge-PMC",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "finetuning_task": "ner",
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 4096,
  "id2label": {
    "0": "B",
    "1": "I",
    "2": "O"
  },
  "initializer_range": 0.01,
  "inner_group_num": 1,
  "intermediate_size": 16384,
  "label2id": {
    "B": 0,
    "I": 1,
    "O": 2
  },
  "layer_norm_eps": 1e-12,
  "layers_to_keep": [],
  "max_position_embeddings": 512,
  "model_type": "albert

In [ ]:
!python3 run_flax_ner.py --model_name_or_path sultan/BioM-ALBERT-xxlarge-PMC \
--train_file data/tokcls/JNLPBA_hf/train.json \
--validation_file data/tokcls/JNLPBA_hf/dev.json \
--test_file data/tokcls/JNLPBA_hf/test.json \
--do_train \
--do_eval \
--do_test \
--per_device_train_batch_size 3 \
--learning_rate 5e-5 \
--num_train_epochs 20 \
--output_dir out \
--overwrite_output_dir \
--blurb_task JNLPBA \
--max_seq_length 512 \
--return_entity_level_metrics \
--eval_steps 1000

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 711.99it/s]
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ALBERT-xxlarge-PMC/snapshots/047499f199be4e57c5dd131a355914131d9c9669/config.json
Model config AlbertConfig {
  "_name_or_path": "sultan/BioM-ALBERT-xxlarge-PMC",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "finetuning_task": "ner",
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 4096,
  "id2label": {
    "0": "B",
    "1": "I",
    "2": "O"
  },
  "initializer_range": 0.01,
  "inner_group_num": 1,
  "intermediate_size": 16384,
  "label2id": {
    "B": 0,
    "I": 1,
    "O": 2
  },
  "layer_norm_eps": 1e-12,
  "layers_to_keep": [],
  "max_position_embeddings": 512,
  "model_type": "albert

In [ ]:
!python3 run_flax_ner.py --model_name_or_path sultan/BioM-ALBERT-xxlarge-PMC \
--train_file data/tokcls/ebmnlp_hf/train.json \
--validation_file data/tokcls/ebmnlp_hf/dev.json \
--test_file data/tokcls/ebmnlp_hf/test.json \
--do_train \
--do_eval \
--do_test \
--per_device_train_batch_size 2 \
--learning_rate 4e-5 \
--num_train_epochs 1 \
--output_dir out \
--overwrite_output_dir \
--blurb_task "EBM PICO" \
--max_seq_length 512 \
--return_macro_metrics \
--eval_steps 1000

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 636.63it/s]
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ALBERT-xxlarge-PMC/snapshots/047499f199be4e57c5dd131a355914131d9c9669/config.json
Model config AlbertConfig {
  "_name_or_path": "sultan/BioM-ALBERT-xxlarge-PMC",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "finetuning_task": "ner",
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 4096,
  "id2label": {
    "0": "B-INT",
    "1": "B-OUT",
    "2": "B-PAR",
    "3": "O"
  },
  "initializer_range": 0.01,
  "inner_group_num": 1,
  "intermediate_size": 16384,
  "label2id": {
    "B-INT": 0,
    "B-OUT": 1,
    "B-PAR": 2,
    "O": 3
  },
  "layer_norm_eps": 1e-12,
  "layers_to_keep": [],
  "max_

# **BioM-ELECTRA-Base**

In [ ]:
!python3 run_flax_seq.py --model_name_or_path sultan/BioM-ELECTRA-Base-Discriminator \
--train_file data/seqcls/chemprot_hf/train.json \
--validation_file data/seqcls/chemprot_hf/dev.json \
--test_file data/seqcls/chemprot_hf/test.json \
--do_train \
--do_eval \
--per_device_train_batch_size 1 \
--per_device_eval_batch_size 8 \
--learning_rate 2e-5 \
--num_train_epochs 4 \
--max_seq_length 256 \
--save_strategy no \
--evaluation_strategy no \
--output_dir out \
--overwrite_output_dir \
--save_strategy no \
--evaluation_strategy no \
--eval_steps 100000 \
--metric_name PRF1 \
--blurb_task ChemProt

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 668.49it/s]
['0', 'CPR:3', 'CPR:4', 'CPR:5', 'CPR:6', 'CPR:9']
Downloading: 100%|██████████████████████████████| 665/665 [00:00<00:00, 662kB/s]
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ELECTRA-Base-Discriminator/snapshots/8bcc387785592aec3de94134a0c9db5ef6b633e6/config.json
Model config ElectraConfig {
  "_name_or_path": "sultan/BioM-ELECTRA-Base-Discriminator",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
  

In [ ]:
!python3 run_flax_seq.py --model_name_or_path sultan/BioM-ELECTRA-Base-Discriminator  \
--train_file data/seqcls/DDI_hf/train.json \
--validation_file data/seqcls/DDI_hf/dev.json \
--test_file data/seqcls/DDI_hf/test.json \
--do_train \
--do_eval \
--per_device_train_batch_size 3 \
--per_device_eval_batch_size 8 \
--learning_rate 2e-5 \
--num_train_epochs 3 \
--max_seq_length 256 \
--save_strategy no \
--evaluation_strategy no \
--output_dir out \
--overwrite_output_dir \
--save_strategy no \
--evaluation_strategy no \
--eval_steps 100000 \
--metric_name PRF1 \
--blurb_task DDI

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 688.83it/s]
['0', 'DDI-advise', 'DDI-effect', 'DDI-int', 'DDI-mechanism']
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ELECTRA-Base-Discriminator/snapshots/8bcc387785592aec3de94134a0c9db5ef6b633e6/config.json
Model config ElectraConfig {
  "_name_or_path": "sultan/BioM-ELECTRA-Base-Discriminator",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embedding

In [ ]:
!python3 run_flax_seq.py --model_name_or_path sultan/BioM-ELECTRA-Base-Discriminator \
--train_file data/seqcls/GAD_hf/train.json \
--validation_file data/seqcls/GAD_hf/dev.json \
--test_file data/seqcls/GAD_hf/test.json \
--do_train \
--do_eval \
--per_device_train_batch_size 2 \
--per_device_eval_batch_size 8 \
--learning_rate 1e-5 \
--num_train_epochs 4 \
--max_seq_length 256 \
--save_strategy no \
--evaluation_strategy no \
--output_dir out \
--overwrite_output_dir \
--save_strategy no \
--evaluation_strategy no \
--eval_steps 100000 \
--metric_name PRF1 \
--blurb_task GAD

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 851.23it/s]
['0', '1']
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ELECTRA-Base-Discriminator/snapshots/8bcc387785592aec3de94134a0c9db5ef6b633e6/config.json
Model config ElectraConfig {
  "_name_or_path": "sultan/BioM-ELECTRA-Base-Discriminator",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transf

In [ ]:
!python3 run_flax_seq.py --model_name_or_path  sultan/BioM-ELECTRA-Base-Discriminator  \
--train_file data/seqcls/bioasq_hf/train.json \
--validation_file data/seqcls/bioasq_hf/dev.json \
--test_file data/seqcls/bioasq_hf/test.json \
--do_train \
--do_eval \
--per_device_train_batch_size 1 \
--per_device_eval_batch_size 8 \
--learning_rate 3e-5 \
--num_train_epochs 20 \
--max_seq_length 512 \
--save_strategy no \
--evaluation_strategy no \
--output_dir out \
--overwrite_output_dir \
--save_strategy no \
--evaluation_strategy no \
--eval_steps 100000 \
--blurb_task BioASQ

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 843.13it/s]
['no', 'yes']
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ELECTRA-Base-Discriminator/snapshots/8bcc387785592aec3de94134a0c9db5ef6b633e6/config.json
Model config ElectraConfig {
  "_name_or_path": "sultan/BioM-ELECTRA-Base-Discriminator",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "tra

In [ ]:
!python3 run_flax_seq.py --model_name_or_path sultan/BioM-ELECTRA-Base-Discriminator \
--train_file data/seqcls/pubmedqa_hf/train.json \
--validation_file data/seqcls/pubmedqa_hf/dev.json \
--test_file data/seqcls/pubmedqa_hf/test.json \
--do_train \
--do_eval \
--per_device_train_batch_size 3 \
--per_device_eval_batch_size 8 \
--learning_rate 3e-5 \
--num_train_epochs 30 \
--max_seq_length 512 \
--save_strategy no \
--evaluation_strategy no \
--output_dir out \
--overwrite_output_dir \
--save_strategy no \
--evaluation_strategy no \
--eval_steps 100000 \
--blurb_task PubMedQA

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 749.21it/s]
['maybe', 'no', 'yes']
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ELECTRA-Base-Discriminator/snapshots/8bcc387785592aec3de94134a0c9db5ef6b633e6/config.json
Model config ElectraConfig {
  "_name_or_path": "sultan/BioM-ELECTRA-Base-Discriminator",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  

In [ ]:
!python3 run_flax_seq.py --model_name_or_path sultan/BioM-ELECTRA-Base-Discriminator \
--train_file data/seqcls/BIOSSES_hf/train.json \
--validation_file data/seqcls/BIOSSES_hf/dev.json \
--test_file data/seqcls/BIOSSES_hf/test.json \
--do_train \
--do_eval \
--per_device_train_batch_size 4 \
--per_device_eval_batch_size 8 \
--learning_rate 3e-5 \
--num_train_epochs 30 \
--max_seq_length 512 \
--save_strategy no \
--evaluation_strategy no \
--output_dir out \
--overwrite_output_dir \
--save_strategy no \
--evaluation_strategy no \
--eval_steps 100000 \
--metric_name pearsonr \
--blurb_task BIOSSES

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 900.84it/s]
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ELECTRA-Base-Discriminator/snapshots/8bcc387785592aec3de94134a0c9db5ef6b633e6/config.json
Model config ElectraConfig {
  "_name_or_path": "sultan/BioM-ELECTRA-Base-Discriminator",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.

In [ ]:
!python3 run_flax_ner.py --model_name_or_path sultan/BioM-ELECTRA-Base-Discriminator \
--train_file data/tokcls/BC5CDR-chem_hf/train.json \
--validation_file data/tokcls/BC5CDR-chem_hf/dev.json \
--test_file data/tokcls/BC5CDR-chem_hf/test.json \
--do_train \
--do_eval \
--do_test \
--per_device_train_batch_size 1 \
--learning_rate 4e-5 \
--num_train_epochs 20 \
--output_dir out \
--overwrite_output_dir \
--blurb_task BC5CDR-chem \
--max_seq_length 512 \
--return_entity_level_metrics \
--eval_steps 10000000

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 744.15it/s]
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ELECTRA-Base-Discriminator/snapshots/8bcc387785592aec3de94134a0c9db5ef6b633e6/config.json
Model config ElectraConfig {
  "_name_or_path": "sultan/BioM-ELECTRA-Base-Discriminator",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "finetuning_task": "ner",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "B",
    "1": "I",
    "2": "O"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B": 0,
    "I": 1,
    "O": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "abs

In [ ]:
!python3 run_flax_ner.py --model_name_or_path sultan/BioM-ELECTRA-Base-Discriminator \
--train_file data/tokcls/BC5CDR-disease_hf/train.json \
--validation_file data/tokcls/BC5CDR-disease_hf/dev.json \
--test_file data/tokcls/BC5CDR-disease_hf/test.json \
--do_train \
--do_eval \
--do_test \
--per_device_train_batch_size 1 \
--learning_rate 7e-5 \
--num_train_epochs 7 \
--output_dir out \
--overwrite_output_dir \
--blurb_task BC5-disease \
--max_seq_length 512 \
--return_entity_level_metrics \
--eval_steps 1000

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 687.78it/s]
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ELECTRA-Base-Discriminator/snapshots/8bcc387785592aec3de94134a0c9db5ef6b633e6/config.json
Model config ElectraConfig {
  "_name_or_path": "sultan/BioM-ELECTRA-Base-Discriminator",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "finetuning_task": "ner",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "B",
    "1": "I",
    "2": "O"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B": 0,
    "I": 1,
    "O": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "abs

In [ ]:
!python3 run_flax_ner.py --model_name_or_path sultan/BioM-ELECTRA-Base-Discriminator \
--train_file data/tokcls/BC2GM_hf/train.json \
--validation_file data/tokcls/BC2GM_hf/dev.json \
--test_file data/tokcls/BC2GM_hf/test.json \
--do_train \
--do_eval \
--do_test \
--per_device_train_batch_size 2 \
--learning_rate 4e-5 \
--num_train_epochs 20 \
--output_dir out \
--overwrite_output_dir \
--blurb_task BC2GM \
--max_seq_length 512 \
--return_entity_level_metrics \
--eval_steps 1000

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 706.23it/s]
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ELECTRA-Base-Discriminator/snapshots/8bcc387785592aec3de94134a0c9db5ef6b633e6/config.json
Model config ElectraConfig {
  "_name_or_path": "sultan/BioM-ELECTRA-Base-Discriminator",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "finetuning_task": "ner",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "B",
    "1": "I",
    "2": "O"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B": 0,
    "I": 1,
    "O": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "abs

In [ ]:
!python3 run_flax_ner.py --model_name_or_path sultan/BioM-ELECTRA-Base-Discriminator \
--train_file data/tokcls/NCBI-disease_hf/train.json \
--validation_file data/tokcls/NCBI-disease_hf/dev.json \
--test_file data/tokcls/NCBI-disease_hf/test.json \
--do_train \
--do_eval \
--do_test \
--per_device_train_batch_size 1 \
--learning_rate 7e-5 \
--num_train_epochs 6 \
--output_dir out \
--overwrite_output_dir \
--blurb_task NCBI-disease \
--max_seq_length 512 \
--return_entity_level_metrics \
--eval_steps 1000

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 730.25it/s]
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ELECTRA-Base-Discriminator/snapshots/8bcc387785592aec3de94134a0c9db5ef6b633e6/config.json
Model config ElectraConfig {
  "_name_or_path": "sultan/BioM-ELECTRA-Base-Discriminator",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "finetuning_task": "ner",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "B",
    "1": "I",
    "2": "O"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B": 0,
    "I": 1,
    "O": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "abs

In [ ]:
!python3 run_flax_ner.py --model_name_or_path sultan/BioM-ELECTRA-Base-Discriminator \
--train_file data/tokcls/JNLPBA_hf/train.json \
--validation_file data/tokcls/JNLPBA_hf/dev.json \
--test_file data/tokcls/JNLPBA_hf/test.json \
--do_train \
--do_eval \
--do_test \
--per_device_train_batch_size 1 \
--learning_rate 5e-5 \
--num_train_epochs 2 \
--output_dir out \
--overwrite_output_dir \
--blurb_task JNLPBA \
--max_seq_length 512 \
--return_entity_level_metrics \
--eval_steps 1000

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 705.24it/s]
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ELECTRA-Base-Discriminator/snapshots/8bcc387785592aec3de94134a0c9db5ef6b633e6/config.json
Model config ElectraConfig {
  "_name_or_path": "sultan/BioM-ELECTRA-Base-Discriminator",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "finetuning_task": "ner",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "B",
    "1": "I",
    "2": "O"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B": 0,
    "I": 1,
    "O": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "abs

In [ ]:
!python3 run_flax_ner.py --model_name_or_path sultan/BioM-ELECTRA-Base-Discriminator \
--train_file data/tokcls/ebmnlp_hf/train.json \
--validation_file data/tokcls/ebmnlp_hf/dev.json \
--test_file data/tokcls/ebmnlp_hf/test.json \
--do_train \
--do_eval \
--do_test \
--per_device_train_batch_size 1 \
--learning_rate 1e-5 \
--num_train_epochs 1 \
--output_dir out \
--overwrite_output_dir \
--blurb_task "EBM PICO" \
--max_seq_length 512 \
--return_macro_metrics \
--eval_steps 1000

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 634.12it/s]
loading configuration file config.json from cache at /home/big35manf/.cache/huggingface/hub/models--sultan--BioM-ELECTRA-Base-Discriminator/snapshots/8bcc387785592aec3de94134a0c9db5ef6b633e6/config.json
Model config ElectraConfig {
  "_name_or_path": "sultan/BioM-ELECTRA-Base-Discriminator",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "finetuning_task": "ner",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "B-INT",
    "1": "B-OUT",
    "2": "B-PAR",
    "3": "O"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-INT": 0,
    "B-OUT": 1,
    "B-PAR": 2,
    "O": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "p

# **HoC Results**

Here we the GPU (V100) with Google Colab to generate our results for HoC.

In [ ]:
!nvidia-smi

Mon May 29 23:10:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    22W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone https://github.com/michiyasunaga/LinkBERT

Cloning into 'LinkBERT'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 42 (delta 9), reused 5 (delta 5), pack-reused 29
Unpacking objects: 100% (42/42), 1.49 MiB | 3.90 MiB/s, done.


In [ ]:
!pip install transformers==4.29.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.8 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 5.4 MB/s eta 0:00:00


In [ ]:
!pip3 install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.1 MB/s eta 0:00:00


In [ ]:
!wget https://nlp.stanford.edu/projects/myasu/LinkBERT/data.zip
!unzip  data.zip

--2023-05-29 23:11:18--  https://nlp.stanford.edu/projects/myasu/LinkBERT/data.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 458233246 (437M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 437.00M  21.7MB/s    in 24s     

2023-05-29 23:11:43 (18.5 MB/s) - ‘data.zip’ saved [458233246/458233246]

Archive:  data.zip
   creating: data/
   creating: data/tokcls/
   creating: data/tokcls/BC5CDR-disease_hf/
  inflating: data/tokcls/BC5CDR-disease_hf/test.json  
  inflating: data/tokcls/BC5CDR-disease_hf/train.json  
  inflating: data/tokcls/BC5CDR-disease_hf/dev.json  
   creating: data/tokcls/BC2GM_hf/
  inflating: data/tokcls/BC2GM_hf/train.json  
  inflating: data/tokcls/BC2GM_hf/dev.json  
  inflating: data/tokcls/BC2GM_hf/test.json  
   creating: data/tokcls/ebmnlp_hf/
  inflating: data/tok

BioM-BERT-Large

In [ ]:
!python3 /home/sultan/LinkBERT/src/seqcls/run_seqcls.py --model_name_or_path sultan/BioM-BERT-PubMed-PMC-Large \
--train_file /home/sultan/data/seqcls/hoc_hf/train.json \
--validation_file /home/sultan/data/seqcls/hoc_hf/dev.json \
--test_file /home/sultan/data/seqcls/hoc_hf/test.json \
--do_train \
--do_eval \
--do_predict \
--metric_name hoc \
--per_device_train_batch_size 32 \
--gradient_accumulation_steps 1 \
--learning_rate 3e-5 \
--num_train_epochs 3 \
--warmup_ratio 0 \
--max_seq_length 128 \
--save_strategy no \
--evaluation_strategy no \
--output_dir out \
--overwrite_output_dir \
--fp16

05/29/2023 19:03:50 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: True
05/29/2023 19:03:51 - WARNING - datasets.builder - Found cached dataset json (/root/.cache/huggingface/datasets/json/default-e6daeb8973830076/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
100%|███████████████████████████████████████████| 3/3 [00:00<00:00, 1375.93it/s]
is_multiclass_binary
[WARNING|modeling_utils.py:3175] 2023-05-29 19:03:54,772 >> Some weights of the model checkpoint at sultan/BioM-BERT-PubMed-PMC-Large were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e

In [ ]:
!python3 /home/sultan/LinkBERT/src/seqcls/run_seqcls.py --model_name_or_path sultan/BioM-ELECTRA-Base-Discriminator \
--train_file /home/sultan/data/seqcls/hoc_hf/train.json \
--validation_file /home/sultan/data/seqcls/hoc_hf/dev.json \
--test_file /home/sultan/data/seqcls/hoc_hf/test.json \
--do_train \
--do_eval \
--do_predict \
--metric_name hoc \
--per_device_train_batch_size 24 \
--gradient_accumulation_steps 1 \
--learning_rate 3e-5 \
--num_train_epochs 6 \
--warmup_ratio 0 \
--max_seq_length 128 \
--save_strategy no \
--evaluation_strategy no \
--output_dir out \
--overwrite_output_dir \
--fp16

05/30/2023 04:02:27 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: True
05/30/2023 04:02:28 - WARNING - datasets.builder - Found cached dataset json (/root/.cache/huggingface/datasets/json/default-e6daeb8973830076/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
100%|███████████████████████████████████████████| 3/3 [00:00<00:00, 1319.93it/s]
is_multiclass_binary
[WARNING|modeling_utils.py:3175] 2023-05-30 04:02:30,707 >> Some weights of the model checkpoint at sultan/BioM-ELECTRA-Base-Discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architectu